In [3]:
import pandas as pd
import os
import shutil
import numpy as np
import glob

# from QC.utils import shell_do
# from QC.imputation import *
# import QC.config as config

In [4]:
# # copy files from old GP2 dir to new
# swarm_scripts_dir = f'/data/CARD/PD/GP2/swarm_scripts'
# # plinkfiles_list = glob.glob('/data/CARD/PD/GP2/raw_genotypes/GP2_plink/20*')
# len(plinkfiles_list)
# with open(f'{swarm_scripts_dir}/cp_plinkfiles.swarm', 'w') as f:
#     for plinkfile in plinkfiles_list:
#         cmd = f'cp {plinkfile} /data/GP2/raw_genotypes/ped_bed/'
#         f.write(f'{cmd}\n')
# f.close()


In [5]:
# !swarm -f {swarm_scripts_dir}/cp_plinkfiles.swarm --time=00:15:00

In [ ]:
# check haplotypes in gp2 data per ancestry
geno_dir = '/data/GP2/quality_control/release6/genotype_qc'


In [5]:
old_idat_path = '/data/CARD/PD/GP2/raw_genotypes/GP2_idats'
new_idat_path = '/data/GP2/raw_genotypes/idats'
master_key = '/data/GP2/clinical/master_key/master_key_JAN2023_all_n24603.csv'
key = pd.read_csv(master_key)


,GP2ID,GP2sampleID,manifest_id,phenotype,pheno_for_qc,other_pheno,sex_for_qc,age,age_of_onset,age_at_diagnosis,...,family_history_for_qc,region_for_qc,study,pruned,pruned_reason,label,related,SentrixBarcode_A,SentrixPosition_A,filename
0,PPMI_001005,PPMI_001005_s1,m3,Other,-9,Genetically-enriched-cohort Affected,1,53.0,NaN,NaN,...,Not Reported,DEU,PPMI-EUR,0,NaN,EUR,0,206890350054,R07C01,206890350054_R07C01
1,PPMI_001047,PPMI_001047_s1,m3,Other,-9,Genetidally-enriched-cohort Unaffected,1,58.3,NaN,NaN,...,Not Reported,NOR,PPMI-EUR,0,NaN,EUR,1,206890350054,R08C01,206890350054_R08C01
2,PPMI_000880,PPMI_000880_s1,m3,PD,2,PD,1,68.5,NaN,68.0,...,Not Reported,DEU,PPMI-EUR,0,NaN,EUR,0,206890350004,R01C01,206890350004_R01C01
3,PPMI_000889,PPMI_000889_s1,m3,PD,2,PD,2,54.7,NaN,54.0,...,Not Reported,DEU,PPMI-EUR,0,NaN,EUR,0,206890350004,R02C01,206890350004_R02C01
4,PPMI_000961,PPMI_000961_s1,m3,Other,-9,SWEDD,1,61.8,NaN,62.0,...,Not Reported,AUT,PPMI-EUR,0,NaN,EUR,0,206890350004,R03C01,206890350004_R03C01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24598,MDGAP-EBB_000149,MDGAP-EBB_000149_s1,m4,Control,1,Control,1,NaN,NaN,NaN,...,Not Reported,GBR,MDGAP-EBB,1,callrate_prune,NaN,0,206412050061,R04C01,206412050061_R04C01
24599,MDGAP-EBB_000150,MDGAP-EBB_000150_s1,m4,Control,1,Control,2,NaN,NaN,NaN,...,Not Reported,GBR,MDGAP-EBB,1,callrate_prune,NaN,0,206412050061,R05C01,206412050061_R05C01
24600,MDGAP-EBB_000151,MDGAP-EBB_000151_s1,m4,Control,1,Control,1,NaN,NaN,NaN,...,Not Reported,GBR,MDGAP-EBB,1,callrate_prune,NaN,0,206412050061,R06C01,206412050061_R06C01
24601,MDGAP-EBB_000152,MDGAP-EBB_000152_s1,m4,Control,1,Control,2,NaN,NaN,NaN,...,Not Reported,GBR,MDGAP-EBB,1,callrate_prune,NaN,0,206412050061,R07C01,206412050061_R07C01


In [17]:
# check if all files from master key are present in new idat dir
missing_new = []
missing_old = []
missing_both = []
exists_new = []
exists_old = []

for filename in key.filename:
    code = filename.split('_')[0]
    new_idat_path = f'/data/GP2/raw_genotypes/idats/{code}/{filename}'
    old_idat_path = f'/data/CARD/PD/GP2/raw_genotypes/GP2_idats/{code}/{filename}'
    old_red = f'{old_idat_path}_Red.idat'
    old_grn = f'{old_idat_path}_Grn.idat'
    new_red = f'{new_idat_path}_Red.idat'
    new_grn = f'{new_idat_path}_Grn.idat'
    
    if (os.path.isfile(new_red) & os.path.isfile(new_grn)):
        exists_new.append(filename)
    else:
        missing_new.append(filename)
    
    if (os.path.isfile(old_red) & os.path.isfile(old_grn)):
        exists_old.append(filename)
    else:
        missing_old.append(filename)
    
    
        
        
#     if not (os.path.isfile(old_red) & os.path.isfile(old_grn) & os.path.isfile(new_red) & os.path.isfile(new_grn)):
#         missing_both.append(filename)
        
#     elif (os.path.isfile(new_red) & os.path.isfile(new_grn)) and not (os.path.isfile(old_red) & os.path.isfile(old_grn)):
#         missing_old.append(filename)
    
#     elif (os.path.isfile(old_red) & os.path.isfile(old_grn)) and not (os.path.isfile(new_red) & os.path.isfile(new_grn)):
#         missing_new.append(filename)
    

In [18]:
print(len(missing_new))
print(len(missing_old))
# print(len(missing_both))

607
2797


In [10]:

def compare_directories(dir1, dir2):
    # Get all the files in dir1
    files1 = []
    for root, dirs, files in os.walk(dir1):
        for file in files:
            files1.append(file)
            
    # Get all the files in dir2
    files2 = []
    for root, dirs, files in os.walk(dir2):
        for file in files:
            files2.append(file)
    
    
        # Find the files that differ between the two directories
    only_in_dir1 = set(files1) - set(files2)
    only_in_dir2 = set(files2) - set(files1)
    
    return list(only_in_dir1), list(only_in_dir2)


def find_empty_and_non_empty_dirs(directories):
    empty_dirs = []
    non_empty_dirs = []
    for directory in directories:
        if os.path.isdir(directory):
            if len(os.listdir(directory)) == 0:
                empty_dirs.append(directory)
            else:
                non_empty_dirs.append(directory)
        else:
            print(directory, "is not a directory")
    return (empty_dirs, non_empty_dirs)



In [11]:
old_only_list, new_only_list = compare_directories(old_idat_path, new_idat_path)


In [12]:
print(len(old_only_list))
print(len(new_only_list))

444
4380


In [24]:
!ls -lh /data/CARD/PD/GP2/raw_genotypes/GP2_idats/temp/GP2_Turkey_PPMI_Combo/206412050072

total 522M
-rwxrwxrw-+ 1 levineks levineks 33M Aug 13  2022 206412050072_R01C01_Grn.idat
-rwxrwxrw-+ 1 levineks levineks 33M Aug 13  2022 206412050072_R01C01_Red.idat
-rwxrwxrw-+ 1 levineks levineks 33M Aug 13  2022 206412050072_R02C01_Grn.idat
-rwxrwxrw-+ 1 levineks levineks 33M Aug 13  2022 206412050072_R02C01_Red.idat
-rwxrwxrw-+ 1 levineks levineks 33M Aug 13  2022 206412050072_R03C01_Grn.idat
-rwxrwxrw-+ 1 levineks levineks 33M Aug 13  2022 206412050072_R03C01_Red.idat
-rwxrwxrw-+ 1 levineks levineks 33M Aug 13  2022 206412050072_R04C01_Grn.idat
-rwxrwxrw-+ 1 levineks levineks 33M Aug 13  2022 206412050072_R04C01_Red.idat
-rwxrwxrw-+ 1 levineks levineks 33M Aug 13  2022 206412050072_R05C01_Grn.idat
-rwxrwxrw-+ 1 levineks levineks 33M Aug 13  2022 206412050072_R05C01_Red.idat
-rwxrwxrw-+ 1 levineks levineks 33M Aug 13  2022 206412050072_R06C01_Grn.idat
-rwxrwxrw-+ 1 levineks levineks 33M Aug 13  2022 206412050072_R06C01_Red.idat
-rwxrwxrw-+ 1 levineks levineks 33M Aug 13  2022 2064

In [21]:
!ls /data/GP2/raw_genotypes/idats/temp

In [12]:
old_list = sorted([1,2,3,4])
new_list = sorted([3,2,1])
print(old_list == new_list)
print(set(old_list) - set(new_list))


False
{4}


In [32]:
count = 0
empty = []
old_not_new = []
new_not_old = []

total_idat_count_old = 0
total_idat_count_new = 0

for barcode_path in glob.glob(f'{old_idat_path}/20*'):
    
    count += 1
    # if count > 4:
    #     break
    
    code = barcode_path.split('/')[-1]

    old_list = sorted([x.split('/')[-1] for x in glob.glob(f'{old_idat_path}/{code}/*.idat')])
    new_list = sorted([x.split('/')[-1] for x in glob.glob(f'{new_idat_path}/{code}/*.idat')])
    
    total_idat_count_old += len(old_list)
    total_idat_count_new += len(new_list)
    
    if old_list != new_list:
        diff1 = set(old_list) - set(new_list)
        diff2 = set(new_list) - set(old_list)
        
        diff_old_list = [f'{old_idat_path}/{code}/{x}' for x in diff1]
        diff_new_list = [f'{new_idat_path}/{code}/{x}' for x in diff2]
        
        old_not_new.append(list(diff1))
        new_not_old.append(list(diff2))

print(count)        

3190


In [33]:
total_idat_count_old

45014

In [34]:
total_idat_count_new

45014

In [28]:
45014

45014

In [23]:
cmd = f'rsync -av --exclude={new_idat_path} {old_idat_path}/ {new_idat_path}'
print(cmd)
!swarm -f rsync_idats.swarm -g 32 -t 16 --time=10:00:00

rsync -av --exclude=/data/GP2/raw_genotypes/idats /data/CARD/PD/GP2/raw_genotypes/GP2_idats/ /data/GP2/raw_genotypes/idats
60598970


In [18]:
old_idat_path

'/data/CARD/PD/GP2/raw_genotypes/GP2_idats'

# setup

In [2]:
idat_path = '/data/CARD/PD/GP2/raw_genotypes/GP2_idats'
swarm_scripts_dir = f'/data/CARD/PD/GP2/swarm_scripts'

ilmn_files_path = '/data/CARD/PD/GP2/ilmn_files'
# A1 is hg19, A2 is hg38. csv needed for indel calls
bpm_csv = f'{ilmn_files_path}/NeuroBooster_20042459_A2.csv' 
bpm = f'{ilmn_files_path}/NeuroBooster_20042459_A2.bpm'
egt = f'{ilmn_files_path}/recluster_09272022.egt'
iaap = f'{ilmn_files_path}/iaap-cli/iaap-cli'

ref_fasta = '/data/vitaled2/ref/GRCh38/GCA_000001405.15_GRCh38_no_alt_analysis_set.fna'

chroms = [str(i) for i in range(1,23)] + ['X','Y','M']


key_file = '/data/CARD/PD/GP2/clinical/key_merge/GP2_merge/GP2_master_key_full.txt'
key = pd.read_csv(f'{key_file}', sep='\t')
key.loc[:,'filename'] = key.loc[:,'SentrixBarcode_A'].astype(str) + '_' + key.loc[:,'SentrixPosition_A'].astype(str)
cohorts = [
    'SYNAPS-KZ', 
    'CORIELL', 
    'IPDGCEA-JU', 
    'KOC', 
    'IPDGCAF-NG', 
    'IPDGCEA-MU', 
    'PAGE', 
    'NZP3', 
    'S4', 
    'BCM',
    'BLAACPD-RUSH', 
    'BIOFIND', 
    'UMD', 
    'BLAACPD-UAB', 
    'LCC', 
    'SYNAPS-GR', 
    'MDGAP-QSBB', 
    'APGS', 
    'MDGAP-KINGS', 
    'MDGAP-IBB', 
    'BLAACPD-UC', 
    'BLAACPD-KPM', 
    'PPMI', 
    'IPDGCEA-UKM'
]

release3_cohorts = [
    'APGS', 
    'BCM', 
    'BIOFIND', 
    'BLAACPD-KPM', 
    'BLAACPD-RUSH', 
    'BLAACPD-UAB', 
    'BLAACPD-UC', 
    'CORIELL', 
    'KOC', 
    'LCC', 
    'MDGAP-KINGS', 
    'MDGAP-QSBB', 
    'NZP3', 
    'PAGE', 
    'PPMI', 
    'S4', 
    'SYNAPS-KZ', 
    'UMD'
]

release_key = key.loc[key['study'].isin(cohorts)]
release_key.to_csv('/data/CARD/PD/GP2/clinical/key_merge/GP2_merge/release3_key.csv', index=False, header=True)

samples_list = release_key.loc[:,'filename'].unique()
barcodes_list = list(set([x.split('_')[0] for x in samples_list]))

genes = pd.read_csv('/data/CARD/PD/GP2/ref_panel/glist-hg38', sep='\s+', header=None, names=['chr','start','end','symbol'], dtype={'chr':str,'start':int,'end':int})
genes.columns = ['CHR','START','STOP','NAME']
genes_out = genes.loc[genes.CHR.isin(chroms)]
genes_out[['NAME','CHR','START','STOP']].to_csv('/data/CARD/PD/GP2/ref_panel/glist_hg38_intervals.csv', index=False)
# gene_list originally from plink glist
gene_list = '/data/CARD/PD/GP2/ref_panel/glist_hg38_intervals.csv'

raw_geno_path = '/data/CARD/PD/GP2/raw_genotypes'
snp_metrics_path = f'{raw_geno_path}/GP2_snp_metrics'
idat_path = f'{raw_geno_path}/GP2_idats'
plink_file_path = f'{raw_geno_path}/GP2_plink'

/tmp/ipykernel_68951/78309220.py:17: DtypeWarning: Columns (36,37) have mixed types. Specify dtype option on import or set low_memory=False.
  key = pd.read_csv(f'{key_file}', sep='\t')


In [3]:
len(samples_list)

21895

In [4]:
# copy meta data to release bucket
release3_key = key.loc[key['study'].isin(release3_cohorts)]
release3_key

,GP2sampleID,SentrixBarcode_A,SentrixPosition_A,study,sample_id,sample_type,DNA_volume,DNA_conc,r260_280,Plate_name,...,SampleRepNo,time_from_onset_years,alternative_id3,alternative_id4,Unnamed: 0.1,Unnamed: 0,FID,IID,pheno,filename
0,SYNAPS-KZ_000001_s1,206046190013,R01C01,SYNAPS-KZ,102336,DNA,30,50.0,NaN,Plate 1,...,s1,NaN,NaN,NaN,NaN,NaN,0,206046190013_R01C01,1,206046190013_R01C01
1,SYNAPS-KZ_000002_s1,206046190013,R02C01,SYNAPS-KZ,102337,DNA,30,50.0,NaN,Plate 1,...,s1,NaN,NaN,NaN,NaN,NaN,0,206046190013_R02C01,1,206046190013_R02C01
2,SYNAPS-KZ_000003_s1,206046190013,R03C01,SYNAPS-KZ,102338,DNA,30,50.0,NaN,Plate 1,...,s1,NaN,NaN,NaN,NaN,NaN,0,206046190013_R03C01,1,206046190013_R03C01
3,SYNAPS-KZ_000004_s1,206046190013,R04C01,SYNAPS-KZ,102339,DNA,30,50.0,NaN,Plate 1,...,s1,NaN,NaN,NaN,NaN,NaN,0,206046190013_R04C01,1,206046190013_R04C01
4,SYNAPS-KZ_000005_s1,206046190013,R05C01,SYNAPS-KZ,102341,DNA,30,50.0,NaN,Plate 1,...,s1,NaN,NaN,NaN,NaN,NaN,0,206046190013_R05C01,1,206046190013_R05C01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24406,PPMI_000789_s1,206430130040,R03C01,PPMI,42171197,DNA from whole blood,30.0,73.3,1.880,7000212060,...,s1,NaN,NaN,NaN,NaN,NaN,0,206430130040_R03C01,-9,206430130040_R03C01
24407,PPMI_000781_s1,206430130040,R04C01,PPMI,42171209,DNA from whole blood,30.0,51.95,1.885,7000212060,...,s1,NaN,NaN,NaN,NaN,NaN,0,206430130040_R04C01,-9,206430130040_R04C01
24408,PPMI_000818_s1,206430130040,R05C01,PPMI,42171221,DNA from whole blood,30.0,54.7,1.875,7000212060,...,s1,NaN,NaN,NaN,NaN,NaN,0,206430130040_R05C01,-9,206430130040_R05C01
24409,PPMI_000841_s1,206430130040,R06C01,PPMI,42171233,DNA from whole blood,30.0,75.85,1.885,7000212060,...,s1,NaN,NaN,NaN,NaN,NaN,0,206430130040_R06C01,-9,206430130040_R06C01


In [8]:
qc1

,step,pruned_count,metric,ancestry,level,pass
0,callrate_prune,931,outlier_count,all,sample,True
1,sex_prune,261,outlier_count,all,sample,True
2,het_prune,0,outlier_count,EUR,sample,True
3,het_prune,0,outlier_count,MDE,sample,False
4,het_prune,0,outlier_count,AJ,sample,True
...,...,...,...,...,...,...
77,variant_prune,0,geno_removed_count,FIN,variant,False
78,variant_prune,0,mis_removed_count,FIN,variant,False
79,variant_prune,0,haplotype_removed_count,FIN,variant,False
80,variant_prune,0,hwe_removed_count,FIN,variant,False


In [9]:
metrics1 = '/data/CARD/PD/GP2/genotypes/GP2/round3/clean/GP2_round3_OCT_PART1.QC.metrics.h5'
metrics2 = '/data/CARD/PD/GP2/genotypes/GP2/round3/clean/GP2_round3_OCT_PART2.QC.metrics.h5'

# combine QC metrics
qc1 = pd.read_hdf(metrics1, key='QC')
qc2 = pd.read_hdf(metrics2, key='QC')
qc_merge = qc1.merge(qc2, on=['step','ancestry','metric'])
qc_merge.loc[:,'pruned_count'] = qc_merge.loc[:,'pruned_count_x'] + qc_merge.loc[:,'pruned_count_y']
qc_merge.loc[:,'pass'] = qc_merge.loc[:,'pass_y']
qc_merge.loc[qc_merge.pass_x!=qc_merge.pass_y, 'pass'] = True
qc_merge.rename(columns={'level_x':'level'}, inplace=True)
qc_merge[['step','pruned_count','metric','ancestry','level','pass']].to_csv('/data/CARD/PD/GP2/genotypes/GP2/round3/clean/qc_metrics.csv', index=False, header=True)

# pruned samples- report in master key with clinical data
pruned_samples1 = pd.read_hdf(metrics1, key='pruned_samples')
pruned_samples2 = pd.read_hdf(metrics2, key='pruned_samples')
pruned1 = pruned_samples1.loc[pruned_samples1['FID']!='#FID']
pruned2 = pruned_samples2.loc[pruned_samples2['FID']!='#FID']
pruned_out = pd.concat([pruned1, pruned2], ignore_index=True)
pruned_out.to_csv('/data/CARD/PD/GP2/genotypes/GP2/round3/clean/pruned_samples.csv', index=False, header=True)

# labels- this is EVERYTHING that was run in QC, not just release3 samples-- get from key
labels_df1 = pd.read_hdf(metrics1, key='ancestry_counts')
labels_df2 = pd.read_hdf(metrics2, key='ancestry_counts')
labels_merge = labels_df1.merge(labels_df2, how='left', on='label')
labels_merge.fillna(0, inplace=True)
labels_merge.loc[:,'count_y'] = labels_merge.loc[:,'count_y'].astype(int)
labels_merge.loc[:,'count'] = labels_merge.loc[:,'count_x'] + labels_merge.loc[:,'count_y']

# confusion matrix- just use better of the two. next release run as one set instead of two halves
conf_mat1 = pd.read_hdf(metrics1, key='confusion_matrix', index=True)
conf_mat2 = pd.read_hdf(metrics2, key='confusion_matrix', index=True)
conf_mat1.to_csv('/data/CARD/PD/GP2/genotypes/GP2/round3/clean/confusion_matrix.csv', index=False, header=True)

# pcs
ref_pcs1 = pd.read_hdf(metrics1, key='ref_pcs')
ref_pcs1.to_csv('/data/CARD/PD/GP2/genotypes/GP2/round3/clean/reference_pcs.csv', index=False, header=True)
projected_pcs1 = pd.read_hdf(metrics1, key='projected_pcs')
projected_pcs2 = pd.read_hdf(metrics2, key='projected_pcs')
projected_pcs_out = pd.concat([projected_pcs1,projected_pcs2], ignore_index=True)
projected_pcs_final = projected_pcs_out.loc[projected_pcs_out['IID'].isin(release3_key['GP2sampleID'])]
projected_pcs_final.to_csv('/data/CARD/PD/GP2/genotypes/GP2/round3/clean/projected_pcs.csv', index=False, header=True)

In [11]:
qc_metrics_files = ['/data/CARD/PD/GP2/genotypes/GP2/round3/clean/qc_metrics.csv',
'/data/CARD/PD/GP2/genotypes/GP2/round3/clean/confusion_matrix.csv', 
'/data/CARD/PD/GP2/genotypes/GP2/round3/clean/projected_pcs.csv',
'/data/CARD/PD/GP2/genotypes/GP2/round3/clean/reference_pcs.csv',
'/data/CARD/PD/GP2/genotypes/GP2/round3/clean/ancestry_table.csv',
'/data/CARD/PD/GP2/genotypes/GP2/round3/clean/related_samples.csv'
]

for f in qc_metrics_files:
    !module load google-cloud-sdk/397.0.0; gsutil cp {f} gs://gp2tier2/release3_31102022/meta_data/qc_metrics/

[+] Loading python 3.7  ... 
[+] Loading google-cloud-sdk  397.0.0 


Updates are available for some Google Cloud CLI components.  To install them,
please run:
  $ gcloud components update

Copying file:///data/CARD/PD/GP2/genotypes/GP2/round3/clean/qc_metrics.csv [Content-Type=text/csv]...
/ [1 files][  3.8 KiB/  3.8 KiB]                                                
Operation completed over 1 objects/3.8 KiB.                                      


In [116]:
!module load google-cloud-sdk/397.0.0; gsutil cp /data/CARD/PD/GP2/genotypes/GP2/round3/clean/master_key_release3_final.csv gs://gp2tier2/release3_31102022/clinical_data/

[-] Unloading python 3.8  ... 
[+] Loading python 3.7  ... 
[+] Loading google-cloud-sdk  397.0.0 

The following have been reloaded with a version change:
  1) python/3.8 => python/3.7

Copying file:///data/CARD/PD/GP2/genotypes/GP2/round3/clean/master_key_release3_final.csv [Content-Type=text/csv]...
- [1 files][  1.4 MiB/  1.4 MiB]                                                
Operation completed over 1 objects/1.4 MiB.                                      


In [106]:
# now pull release2 samples
ancestry_labels = [x.split('/')[-1].replace('.bed','').split('_')[-1] for x in glob.glob('/data/CARD/PD/GP2/genotypes/GP2/round3/clean/GP2_round3_NEW_MERGE*.bed') if '_maf_hwe' not in x]
for label in ancestry_labels:
    release2_fam = f'/data/CARD/PD/GP2/genotypes/GP2/round2/clean/GP2_round2_{label}_maf_hwe.fam'
    
    if os.path.isfile(release2_fam):
        print(label)
        print()
        fam = pd.read_csv(release2_fam, sep='\s+', header=None, names=['fid','iid','pat','mat','sex','pheno'], usecols=['fid','iid'])
        fam.to_csv(f'/data/CARD/PD/GP2/genotypes/GP2/round3/clean/release2_{label}.samples', sep='\t', header=False, index=False)
    else:
        print(f'{label} NOT IN RELEASE2')
        print()

AMR

EUR

SAS

MDE NOT IN RELEASE2

EAS

AAC

AFR

CAS

AJ



In [111]:
!module load google-cloud-sdk/397.0.0; gsutil cp /data/CARD/PD/GP2/genotypes/GP2/round3/clean/release2_*.samples gs://gp2tier2/release3_31102022/meta_data/previous_release_samples/

[-] Unloading python 3.8  ... 
[+] Loading python 3.7  ... 
[+] Loading google-cloud-sdk  397.0.0 

The following have been reloaded with a version change:
  1) python/3.8 => python/3.7

Copying file:///data/CARD/PD/GP2/genotypes/GP2/round3/clean/release2_AAC.samples [Content-Type=application/octet-stream]...
Copying file:///data/CARD/PD/GP2/genotypes/GP2/round3/clean/release2_AFR.samples [Content-Type=application/octet-stream]...
Copying file:///data/CARD/PD/GP2/genotypes/GP2/round3/clean/release2_AJ.samples [Content-Type=application/octet-stream]...
Copying file:///data/CARD/PD/GP2/genotypes/GP2/round3/clean/release2_AMR.samples [Content-Type=application/octet-stream]...
\ [4 files][ 57.8 KiB/ 57.8 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be 

In [72]:
# get missing idats
exists_idats = []
missing_idats = []
for sample in release3_key.IID.unique():
    code = sample.split('_')[0]
    grn = f'{idat_path}/{code}/{sample}_Grn.idat'
    red = f'{idat_path}/{code}/{sample}_Red.idat'
    if os.path.isfile(grn) & os.path.isfile(red):
        exists_idats.append(sample)
    else:
        missing_idats.append(sample)

exists_peds = []
missing_peds = []
for sample in exists_idats:
    code = sample.split('_')[0]
    ped = f'{plink_file_path}/{sample}.ped'
    if os.path.isfile(ped):
        exists_peds.append(sample)
    else:
        missing_peds.append(sample)
        
print(f'n existing idats: {len(exists_idats)}')
print(f'n existing peds: {len(exists_peds)}')
print(f'n missing idats: {len(missing_idats)}')
print(f'n corrupted idats (missing ped): {len(missing_peds)}')
missing_idat_df1 = pd.DataFrame({'IID':missing_idats})
missing_idat_df1.loc[:,'pruned_reason'] = 'missing_idat'
missing_idat_df2 = pd.DataFrame({'IID':missing_peds})
missing_idat_df2.loc[:,'pruned_reason'] = 'corrupted_idat'
missing_idat_out = pd.concat([missing_idat_df1,missing_idat_df2], ignore_index=True)
missing_idat_out.to_csv('/data/CARD/PD/GP2/genotypes/GP2/round3/clean/missing_idats.csv', index=False, header=True)   

n existing idats: 16812
n existing peds: 16790
n missing idats: 45
n corrupted idats (missing ped): 22


In [64]:
len(exists_peds)

16790

In [ ]:
# create ped for ALL GP2 samples
with open(f'{swarm_scripts_dir}/idat_to_ped.swarm', 'w') as f:
    
    for code in barcodes_list:
        
        idat_to_ped_cmd = f'\
{iaap} gencall \
{bpm} \
{egt} \
{plink_file_path} \
-f {idat_path}/{code} \
-p \
-t 8'
        
        f.write(f'{idat_to_ped_cmd}\n')
f.close()

# run swarm
!swarm -f {swarm_scripts_dir}/idat_to_ped.swarm -g 32 -t 16 --time=10:00:00 --logdir {swarm_scripts_dir}/logs --gres=lscratch:20 --partition=norm


In [ ]:
# copy map file
count = 0
map_file = f'/data/CARD/PD/GP2/raw_genotypes/GP2_plink/NeuroBooster_20042459_A2.map'

with open(f'{swarm_scripts_dir}/GP2_copy_map.swarm', 'w') as f:
    
    for sample in samples_list:
        
        ped = f'{plink_file_path}/{sample}.ped'
        out_map = f'{plink_file_path}/{sample}.map'
        if os.path.isfile(ped):
            cmd = f'cp {map_file} {out_map}'
            count += 1
            f.write(f'{cmd}\n')
        else:
            pass
    print(count)
f.close()

!swarm -f {swarm_scripts_dir}/GP2_copy_map.swarm --logdir {swarm_scripts_dir}/logs --gres=lscratch:20 --partition=norm

In [ ]:
# create bed
with open(f'{swarm_scripts_dir}/make_bed.swarm', 'w') as f:
    for sample in samples_list:
        ped = f'{plink_file_path}/{sample}'
        make_bed_cmd = f'\
plink \
--file {ped} \
--make-bed \
--out {ped}'

        f.write(f'{make_bed_cmd}\n')
f.close()

# run swarm
!swarm -f {swarm_scripts_dir}/make_bed.swarm -g 16 -t 8 --time=1:00:00 --logdir {swarm_scripts_dir}/logs --module plink/1.9 --gres=lscratch:20 --partition=norm

In [ ]:
# get snp metrics using new cluster file
bcftools_plugins_path = '/data/vitaled2/bin'
with open(f'{swarm_scripts_dir}/snp_metrics.swarm', 'w') as f:
    for code in barcodes_list:
        idat_path_ = f'{idat_path}/{code}'
        metrics_out = f'{snp_metrics_path}'

        cmd = f'\
python /data/vitaled2/GenoTools/run_snp_metrics_pipeline.py \
--idat_path {idat_path_} \
--bpm {bpm} \
--bpm_csv {bpm_csv} \
--egt {egt} \
--ref_fasta {ref_fasta} \
--out_path {metrics_out} \
--iaap {iaap} \
--bcftools_plugins_path {bcftools_plugins_path}'
        f.write(f'{cmd}\n')
f.close()

!swarm -f {swarm_scripts_dir}/snp_metrics.swarm -g 16 -t 16 --time=04:00:00 --logdir {swarm_scripts_dir}/logs --gres=lscratch:20 --partition=norm

In [78]:
# check if snp metrics produced across all samples
completed_list = []
missing_list = []
for sample in samples_list:
    code = sample.split('_')[0]
    metrics_path = f'{snp_metrics_path}/{code}'
    for chrom in chroms:
        mfile = f'{metrics_path}/snp_metrics_{sample}_chr{chrom}.csv'
        if os.path.isfile(mfile):
            completed_list.append(sample)
        else:
            missing_list.append(sample)
completed_final = list(set(completed_list))
missing_final =list(set(missing_list))
        
    
    

In [79]:
print(len(completed_final))
print(len(missing_final))

21748
147


In [81]:
idat_path

'/data/CARD/PD/GP2/raw_genotypes/GP2_idats'

In [84]:
# now check if we have idats for missing snp metrics
missing_idats = []
exists_idats = []
for sample in missing_final:
    code = sample.split('_')[0]
    grn = f'{idat_path}/{code}/{sample}_Grn.idat'
    red = f'{idat_path}/{code}/{sample}_Red.idat'
    if os.path.isfile(grn) & os.path.isfile(red):
        exists_idats.append(sample)
    else:
        missing_idats.append(sample)

In [89]:
print(len(missing_idats))
print(len(exists_idats))

101
46


In [16]:
print('n ped:')
!ls {plink_file_path}/*.ped | wc -l
print()
print('n map:')
!ls {plink_file_path}/*.map | wc -l
print()
print('n bed:')
!ls {plink_file_path}/*.bed | wc -l

n ped:
21940

n map:
21749

n bed:
21750


In [18]:
ped_files_list = glob.glob(f'{plink_file_path}/*.ped')
map_files_list = glob.glob(f'{plink_file_path}/*.map')
bed_files_list = glob.glob(f'{plink_file_path}/*.bed')

In [23]:
ped_samples = [x.split('/')[-1].replace('.ped','') for x in ped_files_list]
map_samples = [x.split('/')[-1].replace('.map','') for x in map_files_list]

In [29]:
missing_maps = [x for x in ped_samples if x not in map_samples]
len(missing_maps)

192

In [37]:
# copy map file for missings
count = 0
map_file = f'/data/CARD/PD/GP2/raw_genotypes/GP2_plink/NeuroBooster_20042459_A2.map'

with open(f'{swarm_scripts_dir}/GP2_copy_map2.swarm', 'w') as f:
    
    for sample in missing_maps:
        
        ped = f'{plink_file_path}/{sample}.ped'
        out_map = f'{plink_file_path}/{sample}.map'
        if os.path.isfile(ped):
            cmd = f'cp {map_file} {out_map}'
            !{cmd}
            count += 1
            f.write(f'{cmd}\n')
        else:
            print(f'{ped} does not exist!!!')
    print(count)
f.close()

# just run in cell with small number of missings
# !swarm -f {swarm_scripts_dir}/GP2_copy_map2.swarm --logdir {swarm_scripts_dir}/logs --gres=lscratch:20 --partition=norm

192


In [40]:
for sample in missing_maps:
    ped = f'{plink_file_path}/{sample}'
#     !module load plink/1.9; plink --file {ped} --make-bed --out {ped}
    

In [39]:
# check counts again
print('n ped:')
!ls {plink_file_path}/*.ped | wc -l
print()
print('n map:')
!ls {plink_file_path}/*.map | wc -l
print()
print('n bed:')
!ls {plink_file_path}/*.bed | wc -l

n ped:
21940

n map:
21941

n bed:
21942


# CNVs

In [5]:
cnv_path = f'/data/CARD/PD/GP2/raw_genotypes/GP2_cnvs/release3'
# using just autosomes for CNV analysis due to call quality
chroms = [str(i) for i in range(1,23)]

ancestry_labels = [x.split('/')[-1].replace('.bed','').split('_')[-1] for x in glob.glob('/data/CARD/PD/GP2/genotypes/GP2/round3/clean/GP2_round3_NEW_MERGE*.bed') if '_maf_hwe' not in x]

release_key[['FID','GP2sampleID']].to_csv(f'{cnv_path}/release3.samples', sep='\t', header=False, index=False)
release_key[['GP2sampleID','IID']].to_csv(f'{cnv_path}/release3_sample_id_key.csv')
release_covars = release_key.loc[:,['FID', 'GP2sampleID','sex_for_qc', 'age', 'age_of_onset']]

In [147]:
# snp QC for cnvs
for label in ancestry_labels:
    out_name = f'GP2_round3_NEW_MERGE_{label}'
    out_path = f'{cnv_path}/{out_name}'
    geno_path = f'/data/CARD/PD/GP2/genotypes/GP2/round3/clean/{out_name}'

    cmd1 = f'\
module load plink/1.9; plink \
--bfile {geno_path} \
--keep {cnv_path}/release3.samples \
--make-bed \
--out {out_path}'

    cmd2 = f'\
module load plink/1.9; plink \
--bfile {out_path} \
--maf 0.01 \
--geno 0.02 \
--hwe 5e-6 \
--autosome \
--make-bed \
--out {out_path}_clean'

    cmd3 = f'\
module load plink/1.9; plink \
--bfile {out_path}_clean \
--pca \
--out {out_path}_clean_pcs'

    cmds = [cmd1, cmd2, cmd3]

    for cmd in cmds:
        !{cmd}

    pcs = pd.read_csv(f'{out_path}_clean_pcs.eigenvec', sep='\s+')
    pc_num = pcs.iloc[:, 2:].shape[1]
    pc_names = ['FID','GP2sampleID'] + [f'PC{i}' for i in range(1, pc_num+1)]
    pcs.columns = pc_names

    cov = pcs.merge(release_covars, on=['FID','GP2sampleID'], how='left')
    cov.age.fillna(cov.age.mean(), inplace=True)
    cov.age_of_onset.fillna(cov.age_of_onset.mean(), inplace=True)
    cov.sex_for_qc.fillna(cov.sex_for_qc.median(), inplace=True)
    cov.rename(columns={'GP2sampleID':'sampleid','sex_for_qc':'sex'}, inplace=True)
    cov.to_csv(f'{out_path}_clean.cov', sep='\t', header=True, index=False)

    samples = cov.merge(release_key[['GP2sampleID','IID']], left_on='sampleid', right_on='GP2sampleID', how='left')
    samples['IID'].to_csv(f'{out_path}_clean_barcode.samples', header=False, index=False)

    bim = pd.read_csv(f'{out_path}_clean.bim', sep='\s+', header=None, names=['chr','id','pos','bp','a1','a2'], usecols=['id'])
    bim.to_csv(f'{out_path}_clean.snps', sep='\t', header=False, index=False)
        

        
# skip ld pruning until we have better quality rare calls
# --exclude exclusion_regions_hg38.txt \


#         cmd3 = f'\
# plink \
# --bfile {geno}_release2_maf_geno_hwe \
# --indep-pairwise 1000 10 0.02 \
# --autosome \
# --out {geno}_release2_prune'
    
#         cmd4 = f'\
# plink \
# --bfile {geno}_release2_maf_geno_hwe \
# --extract {geno}_release2_prune.prune.in \
# --make-bed \
# --out {geno}_release2_clean'

[-] Unloading plink  3.6-alpha 
[+] Loading plink  1.9  on cn4250 

The following have been reloaded with a version change:
  1) plink/3.6-alpha => plink/1.9

PLINK v1.90b3.36 64-bit (31 Mar 2016)      https://www.cog-genomics.org/plink2
(C) 2005-2016 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /data/CARD/PD/GP2/raw_genotypes/GP2_cnvs/release3/GP2_round3_NEW_MERGE_AMR.log.
Options in effect:
  --bfile /data/CARD/PD/GP2/genotypes/GP2/round3/clean/GP2_round3_NEW_MERGE_AMR
  --keep /data/CARD/PD/GP2/raw_genotypes/GP2_cnvs/release3/release3.samples
  --make-bed
  --out /data/CARD/PD/GP2/raw_genotypes/GP2_cnvs/release3/GP2_round3_NEW_MERGE_AMR

1547809 MB RAM detected; reserving 773904 MB for main workspace.
1981184 variants loaded from .bim file.
419 people (232 males, 187 females) loaded from .fam.
414 phenotype values loaded from .fam.
--keep: 419 people remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 419 fou

In [40]:
with open(f'{swarm_scripts_dir}/cnvs.swarm', 'w') as f:
    for label in ancestry_labels:
        out_name = f'GP2_round3_NEW_MERGED_{label}'
        out_path = f'{cnv_path}/{out_name}'
#         geno_path = f'/data/CARD/PD/GP2/genotypes/GP2/round3/clean/{out_name}'
        samples = pd.read_csv(f'{out_path}_clean_barcode.samples', header=None, names=['IID'])
        
        for sample in samples.IID.unique():
            code = sample.split('_')[0]
            for chrom in chroms:
            
                mfile = f'{snp_metrics_path}/{code}/snp_metrics_{sample}_chr{chrom}.csv'

                out_file = f'{cnv_path}/CNV_{label}_{sample}_chr{chrom}.csv'
                intervals = f'/data/CARD/PD/GP2/ref_panel/glist_hg38_chr{chrom}.csv'

                cmd = f'\
python /data/vitaled2/GenoTools/run_cnv_pipeline.py \
--metrics {mfile} \
--out_path {out_file} \
--intervals {intervals} \
--min_variants 10 \
--kb_window 250'
            
                f.write(f'{cmd}\n')
f.close()

In [149]:
!swarm -f {swarm_scripts_dir}/cnvs.swarm -g 4 -t 8 --time=00:30:00 --logdir {swarm_scripts_dir}/logs --gres=lscratch:20 --partition=norm

50568328


In [7]:
!ls {cnv_path}/CNV_EUR*_chr2.csv | wc -l
# !cat /data/CARD/PD/GP2/raw_genotypes/GP2_cnvs/release3/CNV_EUR_204620380001_R01C01_chr2.csv | wc -l


12054


In [8]:
# write dosages per chromosome per ancestry per cnv-type
cnv_types = ['PERCENT_BAF_INSERTION','PERCENT_L2R_DELETION','PERCENT_L2R_DUPLICATION']

with open(f'{swarm_scripts_dir}/cnv_dosages.swarm', 'w') as f:
    for label in ancestry_labels:
        for chrom in chroms:

            cnv_files_list = glob.glob(f'{cnv_path}/CNV_{label}_*_chr{chrom}.csv')
            cnv_files_df = pd.DataFrame({'filename':cnv_files_list})
            cnv_files_df.to_csv(f'{cnv_path}/CNV_{label}_chr{chrom}_files.csv', header=False, index=False)

            for cnv_type in cnv_types:
                
                cmd = f'\
python /data/vitaled2/GenoTools/run_cnv_dosage_pipeline.py \
--files {cnv_path}/CNV_{label}_chr{chrom}_files.csv \
--label {label} \
--chrom {chrom} \
--cnv_type {cnv_type} \
--out_path {cnv_path}/CNV_{label}_chr{chrom}_{cnv_type}.csv'
                f.write(f'{cmd}\n')
f.close()
            


In [9]:
!swarm -f {swarm_scripts_dir}/cnv_dosages.swarm -g 32 -t 16 --logdir {swarm_scripts_dir}/logs --gres=lscratch:20 --partition=norm

50634951


In [16]:
with open(f'{swarm_scripts_dir}/update_cnv_ids.swarm','w') as f:
    for label in ancestry_labels:
        for dose in cnv_types:
            for chrom in chroms:

                dosagefile = f'{cnv_path}/CNV_{label}_chr{chrom}_{cnv_type}.csv'
                dosagefile_out = f'{cnv_path}/CNV_{label}_chr{chrom}_{cnv_type}_gp2ids.csv'
                pheno_out = f'{cnv_path}/GP2_round2_{label}_chr{chrom}_{cnv_type}.pheno'
                cmd = f'\
python /data/vitaled2/GenoTools/CNV/update_cnv_ids.py \
--dosagefile {dosagefile} \
--key /data/CARD/PD/GP2/clinical/key_merge/GP2_merge/release3_key.csv \
--pheno_out {pheno_out} \
--out_path {dosagefile_out}'
                f.write(f'{cmd}\n')
f.close()


In [20]:
!swarm -f {swarm_scripts_dir}/update_cnv_ids.swarm -g 8 -t 8 --logdir {swarm_scripts_dir}/logs --gres=lscratch:20 --partition=norm
# !python /data/vitaled2/GenoTools/CNV/update_cnv_ids.py --dosagefile /data/CARD/PD/GP2/raw_genotypes/GP2_cnvs/release3/CNV_AMR_chr1_PERCENT_L2R_DUPLICATION.csv --key /data/CARD/PD/GP2/clinical/key_merge/GP2_merge/release3_key.csv --pheno_out /data/CARD/PD/GP2/raw_genotypes/GP2_cnvs/release3/GP2_round2_AMR_chr1_PERCENT_L2R_DUPLICATION.pheno --out_path /data/CARD/PD/GP2/raw_genotypes/GP2_cnvs/release3/CNV_AMR_chr1_PERCENT_L2R_DUPLICATION_gp2ids.csv

50639165


In [14]:
# !ls {cnv_path}/CNV_*_chr*_*.csv
!ls /data/vitaled2/GenoTools/CNV/update_cnv_ids.py

clean_snp_metrics.py		 __init__.py
ClusterBuster_testing.ipynb	 process_vcf_snps.py
cnv_gene_caller_alpha.py	 __pycache__
cnv.py				 update_cnv_ids.py
GP2_ClusterBuster_datagen.ipynb


In [99]:
!cat /data/CARD/PD/GP2/raw_genotypes/GP2_cnvs/release3/CNV_EUR_chr2_PERCENT_BAF_INSERTION.csv | wc -l

12055


In [103]:
dosage = pd.read_csv('/data/CARD/PD/GP2/raw_genotypes/GP2_cnvs/release3/CNV_EUR_chr2_PERCENT_BAF_INSERTION.csv')

In [107]:
dosage_merge = dosage.merge(release_key[['GP2sampleID','IID']], left_on='sampleid', right_on='IID')
dosage_out = dosage_merge.drop(columns=['sampleid','IID']).set_index('GP2sampleID').reset_index().rename(columns={'GP2sampleID':'sampleid'})

dosage_pheno = dosage_out.merge(release_key.loc[:,['GP2sampleID','pheno']], left_on='sampleid', right_on='GP2sampleID', how='left')
dosage_pheno2 = dosage_pheno.drop(columns=["GP2sampleID"])
dosage_pheno_out = dosage_pheno2.loc[dosage_pheno2.pheno != -9]
dosage_pheno_out.loc[:,'pheno'] = np.where(dosage_pheno_out.pheno == 1, 0, 1)

/tmp/ipykernel_96408/1900774110.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dosage_pheno_out.loc[:,'pheno'] = np.where(dosage_pheno_out.pheno == 1, 0, 1)


In [109]:
dosage_pheno_out.pheno.value_counts()

1    7410
0    4278
Name: pheno, dtype: int64

In [3]:
# paths that will change with each round
# plink genotype
# geno_path = f'{gp2_qc_path}/round1/GP2_QC_round1'
# # qc paths
# out_dir = f'/data/CARD/PD/GP2/genotypes/GP2/round1/clean'
# out_path = f'{out_dir}/GP2_round1'
# # GP2 ref panel path
# ref_dir_path = '/data/LNG/vitaled2/1kgenomes'
# ref_panel = f'{ref_dir_path}/1kg_ashkj_ref_panel_gp2_pruned_hg38'
# ref_labels = f'{ref_dir_path}/ref_panel_ancestry.txt'
# # imputation paths
# imputed_out_dir = '/data/CARD/PD/GP2/genotypes/GP2/round1/imputed'


In [4]:
# grab samples for reclustering: GD + diverse gp2 samples
gd_path = '/data/CARD/PD/GP2/raw_genotypes/GD'
# !cp {gd_path}/idats/IDATs.zip {gd_path}/idats/gd_idats.zip

In [4]:
import random
labels = ['AAC','AFR','AJ','AMR','CAS','EAS','EUR','FIN','SAS']

# labels = ['AAC']
recluster_samples = []
for label in labels:
    samples = f'/data/CARD/PD/GP2/genotypes/GP2/round2/clean/GP2_round2_{label}_release2.samples'
    
    s_list = pd.read_csv(samples, sep='\t', header=None, names=['FID','IID'])
    if s_list.shape[0]>=500:
        r_sample = random.sample(set(s_list.IID.unique()), 500)
        recluster_samples = recluster_samples + r_sample
    else:
        sample_n = s_list.shape[0]
        r_sample = random.sample(set(s_list.IID.unique()), sample_n)
        recluster_samples = recluster_samples + r_sample

In [6]:
recluster_df = pd.DataFrame({'IID':recluster_samples})
recluster_merge = recluster_df.merge(key, how='left', left_on='IID', right_on='GP2sampleID')

for filename in recluster_merge.filename.unique():
    code = filename.split('_')[0]
    idat_path = '/data/CARD/PD/GP2/raw_genotypes/GP2_idats'
    grn = f'{idat_path}/{code}/{filename}_Grn.idat'
    red = f'{idat_path}/{code}/{filename}_Red.idat'
#     !mkdir /data/CARD/PD/GP2/raw_genotypes/GP2_recluster_idats/{code}
#     !cp {grn} /data/CARD/PD/GP2/raw_genotypes/GP2_recluster_idats/{code}/
#     !cp {red} /data/CARD/PD/GP2/raw_genotypes/GP2_recluster_idats/{code}/
#     if os.path.isfile(grn):
#         pass
#     elif os.path.isfile(red):
#         pass
#     else:
#         print(f'{filename} missing!')
#     /data/CARD/PD/GP2/raw_genotypes/GP2_recluster_idats/
    

In [17]:

with open(f'{swarm_scripts_dir}/zip_recluster.swarm', 'w') as f:
    zip_cmd = 'zip -r /data/CARD/PD/GP2/raw_genotypes/GP2_recluster_idats/recluster_idats.zip /data/CARD/PD/GP2/raw_genotypes/GP2_recluster_idats/'
    f.write(zip_cmd)
f.close()

!swarm -f {swarm_scripts_dir}/zip_recluster.swarm --time=10:00:00

41381368


In [129]:
# gwas on qc'd and imputed data to check lambda before release


plink2 = '/data/vitaled2/GenoTools/exec/plink2'
for label in ancestry_labels:
    geno = f'/data/CARD/PD/GP2/genotypes/GP2/round3/clean/GP2_round3_NEW_MERGE_{label}_maf_hwe'

    !{plink2} --bfile {geno} --pca --out {geno}_pca
#     pcs = pd.read_csv(f'{geno}_pca.eigenvec', sep='\s+')
    # only using pcs for covariates right now because sex included in psam
    !{plink2} --bfile {geno} --glm --covar {geno}_pca.eigenvec --out {geno}_assoc         

PLINK v2.00a3LM AVX2 Intel (17 Dec 2021)       www.cog-genomics.org/plink/2.0/
(C) 2005-2021 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /data/CARD/PD/GP2/genotypes/GP2/round3/clean/GP2_round3_NEW_MERGE_AMR_maf_hwe_pca.log.
Options in effect:
  --bfile /data/CARD/PD/GP2/genotypes/GP2/round3/clean/GP2_round3_NEW_MERGE_AMR_maf_hwe
  --out /data/CARD/PD/GP2/genotypes/GP2/round3/clean/GP2_round3_NEW_MERGE_AMR_maf_hwe_pca
  --pca

Start time: Mon Oct 24 19:17:20 2022
1547809 MiB RAM detected; reserving 773904 MiB for main workspace.
Using up to 144 threads (change this with --threads).
419 samples (187 females, 232 males; 419 founders) loaded from
/data/CARD/PD/GP2/genotypes/GP2/round3/clean/GP2_round3_NEW_MERGE_AMR_maf_hwe.fam.
1122358 variants loaded from
/data/CARD/PD/GP2/genotypes/GP2/round3/clean/GP2_round3_NEW_MERGE_AMR_maf_hwe.bim.
1 binary phenotype loaded (232 cases, 182 controls).
Calculating allele frequencies... 11172329354046525864707581879399don

In [11]:
# !ls /data/CARD/PD/GP2/genotypes/GP2/round3/clean/GP2_round3_NEW_MERGE_{label}_maf_hwe_assoc*
# test = pd.read_csv('/data/CARD/PD/GP2/genotypes/GP2/round3/clean/GP2_round3_NEW_MERGE_AJ_maf_hwe_assoc.PHENO1.glm.logistic.hybrid', sep='\s+')
# test.loc[(test['#CHROM'].isin(chroms)) & (test.TEST=='ADD'),'P']
# calculate_inflation(test.loc[(test['#CHROM'].isin(chroms)) & (test.TEST=='ADD'),'P'], )
infl['metrics']['inflation']

1.0371506306636413

In [23]:
from GWAS.gwas import calculate_inflation
ancestry_labels = [x.split('/')[-1].replace('.bed','').split('_')[-1] for x in glob.glob('/data/CARD/PD/GP2/genotypes/GP2/round3/clean/GP2_round3_NEW_MERGE*.bed') if '_maf_hwe' not in x]
raw_lambda_df = pd.DataFrame()
for label in ancestry_labels:
    print(label)
    fam = pd.read_csv(f'/data/CARD/PD/GP2/genotypes/GP2/round3/clean/GP2_round3_NEW_MERGE_{label}_maf_hwe.fam', sep='\s+', header=None, names=['fid','iid', 'pat', 'mat', 'sex','pheno'], dtype={'fid':str})
    geno = f'/data/CARD/PD/GP2/genotypes/GP2/round3/clean/GP2_round3_NEW_MERGE_{label}_maf_hwe'
    raw_geno_assoc = pd.read_csv(f'{geno}_assoc.PHENO1.glm.logistic.hybrid', sep='\s+')
    raw_geno_assoc_clean = raw_geno_assoc.loc[raw_geno_assoc.TEST=='ADD']
    raw_geno_assoc_final = raw_geno_assoc_clean[~raw_geno_assoc_clean.P.isna()]
    ncases = fam.pheno.value_counts()[2]
    ncontrols = fam.pheno.value_counts()[1]
    print(f'cases: {ncases}, controls: {ncontrols}')
    infl1000 = calculate_inflation(raw_geno_assoc_final.P, normalize=True, ncases=ncases, ncontrols=ncontrols)
    infl = calculate_inflation(raw_geno_assoc_final.P)
    tmp = pd.DataFrame({'label':[label],'lambda':[infl['metrics']['inflation']],'lambda1000':[infl1000['metrics']['inflation']],'ncases':[ncases],'ncontrols':[ncontrols]})
    raw_lambda_df = pd.concat([raw_lambda_df,tmp], ignore_index=True)
    print(infl)
    print(infl1000)
    print()
    print()
raw_lambda_df.to_csv(f'/data/CARD/PD/GP2/genotypes/GP2/round3/clean/lambdas.csv', index=False)


AMR


/tmp/ipykernel_65843/8471684.py:8: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_geno_assoc = pd.read_csv(f'{geno}_assoc.PHENO1.glm.logistic.hybrid', sep='\s+')


cases: 232, controls: 182

RUNNING: lambda_calculation


RUNNING: lambda_calculation

{'pass': True, 'step': 'lambda_calculation', 'metrics': {'inflation': 1.0431289993865933}}
{'pass': True, 'step': 'lambda_calculation', 'metrics': {'inflation': 1.2114366917635662}}


EUR


/tmp/ipykernel_65843/8471684.py:8: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_geno_assoc = pd.read_csv(f'{geno}_assoc.PHENO1.glm.logistic.hybrid', sep='\s+')


cases: 7409, controls: 4277

RUNNING: lambda_calculation


RUNNING: lambda_calculation

{'pass': True, 'step': 'lambda_calculation', 'metrics': {'inflation': 1.1058746726549675}}
{'pass': True, 'step': 'lambda_calculation', 'metrics': {'inflation': 1.019522216369401}}


SAS


...

cases: 474, controls: 945

RUNNING: lambda_calculation


RUNNING: lambda_calculation

{'pass': True, 'step': 'lambda_calculation', 'metrics': {'inflation': 1.0275330164348928}}
{'pass': True, 'step': 'lambda_calculation', 'metrics': {'inflation': 1.0436109998449679}}


CAS


/tmp/ipykernel_65843/8471684.py:8: DtypeWarning: Columns (0,9) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_geno_assoc = pd.read_csv(f'{geno}_assoc.PHENO1.glm.logistic.hybrid', sep='\s+')


cases: 115, controls: 194

RUNNING: lambda_calculation


RUNNING: lambda_calculation

{'pass': True, 'step': 'lambda_calculation', 'metrics': {'inflation': 1.0418736050109447}}
{'pass': True, 'step': 'lambda_calculation', 'metrics': {'inflation': 1.289980814620841}}


AJ


/tmp/ipykernel_65843/8471684.py:8: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_geno_assoc = pd.read_csv(f'{geno}_assoc.PHENO1.glm.logistic.hybrid', sep='\s+')


cases: 609, controls: 324

RUNNING: lambda_calculation


RUNNING: lambda_calculation

{'pass': True, 'step': 'lambda_calculation', 'metrics': {'inflation': 1.0371506306636413}}
{'pass': True, 'step': 'lambda_calculation', 'metrics': {'inflation': 1.0878325589642435}}




In [22]:
!cat /data/CARD/PD/GP2/genotypes/GP2/round3/clean/lambdas.csv

label,lambda,lambda1000,ncases,ncontrols
AMR,"{'pass': True, 'step': 'lambda_calculation', 'metrics': {'inflation': 1.0431289993865933}}","{'pass': True, 'step': 'lambda_calculation', 'metrics': {'inflation': 1.2114366917635662}}",232,182
EUR,"{'pass': True, 'step': 'lambda_calculation', 'metrics': {'inflation': 1.1058746726549675}}","{'pass': True, 'step': 'lambda_calculation', 'metrics': {'inflation': 1.019522216369401}}",7409,4277
SAS,"{'pass': True, 'step': 'lambda_calculation', 'metrics': {'inflation': 0.8197570640981234}}","{'pass': True, 'step': 'lambda_calculation', 'metrics': {'inflation': -5.351844756077739}}",67,18
MDE,"{'pass': True, 'step': 'lambda_calculation', 'metrics': {'inflation': 0.8314933964666802}}","{'pass': True, 'step': 'lambda_calculation', 'metrics': {'inflation': -8.219637398708576}}",54,11
EAS,"{'pass': True, 'step': 'lambda_calculation', 'metrics': {'inflation': 1.0558609650134572}}","{'pass': True, 'step': 'lambda_calculation', 'metrics': {'inflation': 1.

In [24]:
!cat /data/CARD/PD/GP2/genotypes/GP2/round3/clean/lambdas.csv

label,lambda,lambda1000,ncases,ncontrols
AMR,1.0431289993865933,1.2114366917635662,232,182
EUR,1.1058746726549675,1.019522216369401,7409,4277
SAS,0.8197570640981234,-5.351844756077739,67,18
MDE,0.8314933964666802,-8.219637398708576,54,11
EAS,1.0558609650134572,1.094291902297474,428,962
AAC,1.0413015918932855,1.1471249877136804,161,1095
AFR,1.0275330164348928,1.0436109998449679,474,945
CAS,1.0418736050109447,1.289980814620841,115,194
AJ,1.0371506306636413,1.0878325589642435,609,324


In [20]:
# now check lambdas for imputed
chroms = [x for x in range(1,23)]

with open(f'{swarm_scripts_dir}/assoc.swarm', 'w') as f:
    for label in ancestry_labels:
        label_path = f'/data/CARD/PD/GP2/genotypes/GP2/round3/imputed/{label}'
        
        fam = pd.read_csv(f'/data/CARD/PD/GP2/genotypes/GP2/round3/clean/GP2_round3_NEW_MERGE_{label}_maf_hwe.fam', sep='\s+', header=None, names=['fid','iid', 'pat', 'mat', 'sex','pheno'], dtype={'fid':str})
        fam.loc[:,'iid_imp'] = fam.loc[:,'fid'] + '_' + fam.loc[:,'iid']
        fam[['iid_imp','iid']].to_csv(f'{label_path}/{label}.update_ids', sep='\t', header=False, index=False)
        fam[['fid','iid','sex']].to_csv(f'{label_path}/{label}.sex', sep='\t', header=False, index=False)
        fam[['fid','iid','pheno']].to_csv(f'{label_path}/{label}.pheno', sep='\t', header=False, index=False)
        
        for chrom in chroms:
            geno = f'{label_path}/chr{chrom}_{label}'
            cmd = f'plink2 --pfile {geno} --update-ids {label_path}/{label}.update_ids --maf 0.005 --hwe 1e-5 --geno 0.01 --make-pgen --out {geno}_prune && \
plink2 --pfile {geno}_prune --update-sex {label_path}/{label}.sex --pheno {label_path}/{label}.pheno --make-pgen --out {geno}_prune_final && \
plink2 --pfile {geno}_prune_final --glm --covar /data/CARD/PD/GP2/genotypes/GP2/round3/clean/GP2_round3_NEW_MERGE_{label}_maf_hwe_pca.eigenvec  --out {geno}_assoc'
# plink2 --pfile {geno}_prune_final --pca --out {geno}_prune_final_pca && \
            f.write(f'{cmd}\n')
f.close()



In [21]:
!swarm -f {swarm_scripts_dir}/assoc.swarm -g 32 -t 32 --time=10:00:00 --logdir {swarm_scripts_dir}/logs --module plink/2.3-alpha --gres=lscratch:20 --partition=norm

51344320


In [19]:
# !cat {swarm_scripts_dir}/logs/swarm_51340828_197.e
# !ls /data/CARD/PD/GP2/genotypes/GP2/round3/clean/GP2_round3_NEW_MERGE_AJ_maf_hwe_final_pca.eigenvec
# !ls /data/CARD/PD/GP2/genotypes/GP2/round3/clean/*.eigenvec
# !ls data/CARD/PD/GP2/genotypes/GP2/round3/clean/GP2_round3_NEW_MERGE_AJ_maf_hwe_final_pca.eigenvec

/data/CARD/PD/GP2/genotypes/GP2/round3/clean/GP2_round3_IPDGCAF_NG_AFR_l444p_regression_clean_pcs.eigenvec
/data/CARD/PD/GP2/genotypes/GP2/round3/clean/GP2_round3_NEW_MERGE_AAC_maf_hwe_pca.eigenvec
/data/CARD/PD/GP2/genotypes/GP2/round3/clean/GP2_round3_NEW_MERGE_AFR_maf_hwe_pca.eigenvec
/data/CARD/PD/GP2/genotypes/GP2/round3/clean/GP2_round3_NEW_MERGE_AJ_maf_hwe_pca.eigenvec
/data/CARD/PD/GP2/genotypes/GP2/round3/clean/GP2_round3_NEW_MERGE_AMR_maf_hwe_pca.eigenvec
/data/CARD/PD/GP2/genotypes/GP2/round3/clean/GP2_round3_NEW_MERGE_CAS_maf_hwe_pca.eigenvec
/data/CARD/PD/GP2/genotypes/GP2/round3/clean/GP2_round3_NEW_MERGE_EAS_maf_hwe_pca.eigenvec
/data/CARD/PD/GP2/genotypes/GP2/round3/clean/GP2_round3_NEW_MERGE_EUR_maf_hwe_pca.eigenvec
/data/CARD/PD/GP2/genotypes/GP2/round3/clean/GP2_round3_NEW_MERGE_MDE_maf_hwe_pca.eigenvec
/data/CARD/PD/GP2/genotypes/GP2/round3/clean/GP2_round3_NEW_MERGE_SAS_maf_hwe_pca.eigenvec


In [25]:
for label in ancestry_labels:
    label_path = f'/data/CARD/PD/GP2/genotypes/GP2/round3/imputed/{label}'
    !head -n 1 {label_path}/chr1_{label}_assoc.PHENO1.glm.logistic.hybrid > {label_path}/{label}.assoc
    for chrom in chroms:
        geno = f'{label_path}/chr{chrom}_{label}_assoc.PHENO1.glm.logistic.hybrid'
        !sed 1,1d {geno} | grep 'ADD' >> {label_path}/{label}.assoc
        
#     geno_assoc = pd.read_csv(geno, sep='\s+')
#     geno_assoc_clean = geno_assoc.loc[geno_assoc.TEST=='ADD']
#     geno_assoc_final = geno_assoc_clean[~geno_assoc_clean.P.isna()]
#     total_assoc_df = pd.concat([total_assoc_df, geno_assoc_final], )

# calculate_inflation(geno_assoc_final.P, normalize=True, ncases=4199, ncontrols=2329)

In [110]:
fam.pheno.value_counts()[-9]

4

In [105]:
fam = pd.read_csv(f'/data/CARD/PD/GP2/genotypes/GP2/round3/clean/GP2_round3_NEW_MERGE_EAS_maf_hwe.fam', sep='\s+', header=None, names=['fid','iid', 'pat', 'mat', 'sex','pheno'], dtype={'fid':str})

In [28]:
ancestry_labels

['AMR', 'EUR', 'SAS', 'MDE', 'EAS', 'AAC', 'AFR', 'CAS', 'AJ']

In [30]:
imputed_lambda_df = pd.DataFrame()

for label in ancestry_labels:
    print(label)
    fam = pd.read_csv(f'/data/CARD/PD/GP2/genotypes/GP2/round3/clean/GP2_round3_NEW_MERGE_{label}_maf_hwe.fam', sep='\s+', header=None, names=['fid','iid', 'pat', 'mat', 'sex','pheno'], dtype={'fid':str})
    
    label_path = f'/data/CARD/PD/GP2/genotypes/GP2/round3/imputed/{label}'
    total_assoc = pd.read_csv(f'{label_path}/{label}.assoc', sep='\s+')
    total_assoc_clean = total_assoc.loc[total_assoc.TEST=='ADD']
    total_assoc_final = total_assoc_clean[~total_assoc_clean.P.isna()]
    ncases = fam.pheno.value_counts()[2]
    ncontrols = fam.pheno.value_counts()[1]
    print(f'cases: {ncases}, controls: {ncontrols}')
    infl1000 = calculate_inflation(total_assoc_final.P, normalize=True, ncases=ncases, ncontrols=ncontrols)
    infl = calculate_inflation(total_assoc_final.P)
    
    tmp = pd.DataFrame({'label':[label],'lambda':[infl['metrics']['inflation']],'lambda1000':[infl1000['metrics']['inflation']],'ncases':[ncases],'ncontrols':[ncontrols]})
    imputed_lambda_df = pd.concat([imputed_lambda_df, tmp], ignore_index=True)
    print('inflation:', infl)
    print('inflation1000:', infl1000)
    print()
    print()
imputed_lambda_df.to_csv(f'/data/CARD/PD/GP2/genotypes/GP2/round3/imputed/imputed_lambdas.csv', index=False)


AMR
cases: 232, controls: 182

RUNNING: lambda_calculation


RUNNING: lambda_calculation

inflation: {'pass': True, 'step': 'lambda_calculation', 'metrics': {'inflation': 1.0377689013934246}}
inflation1000: {'pass': True, 'step': 'lambda_calculation', 'metrics': {'inflation': 1.1851592124961843}}


EUR
cases: 7409, controls: 4277

RUNNING: lambda_calculation


RUNNING: lambda_calculation

inflation: {'pass': True, 'step': 'lambda_calculation', 'metrics': {'inflation': 1.0930145859330114}}
inflation1000: {'pass': True, 'step': 'lambda_calculation', 'metrics': {'inflation': 1.0171509467425899}}


SAS
cases: 67, controls: 18

RUNNING: lambda_calculation


RUNNING: lambda_calculation

inflation: {'pass': True, 'step': 'lambda_calculation', 'metrics': {'inflation': 1.0005879849104475}}
inflation1000: {'pass': True, 'step': 'lambda_calculation', 'metrics': {'inflation': 1.0207208612719902}}


MDE
cases: 54, controls: 11

RUNNING: lambda_calculation


RUNNING: lambda_calculation

inflation: {

In [31]:
imputed_lambda_df

,label,lambda,lambda1000,ncases,ncontrols
0,AMR,1.037769,1.185159,232,182
1,EUR,1.093015,1.017151,7409,4277
2,SAS,1.000588,1.020721,67,18
3,MDE,1.213926,12.704727,54,11
4,EAS,1.061519,1.103842,428,962
5,AAC,1.006561,1.023373,161,1095
6,AFR,1.021862,1.034629,474,945
7,CAS,1.053843,1.372868,115,194
8,AJ,1.036173,1.085522,609,324


In [4]:
labels_df = pd.read_csv(ref_labels, sep='\t', header=None, names=['fid','iid','label'])
labels_df.label.value_counts()

EAS    504
AFR    504
SAS    489
AJ     471
EUR    404
AMR    347
AAC    157
FIN     99
Name: label, dtype: int64

In [40]:
for sample in samples_list:
        geno = f'/data/CARD/PD/GP2/raw_genotypes/GP2_plink/{sample}'
        if os.path.isfile(f'{geno}.ped'):
            pass
        else:
            print(f'{geno} doesnt exist')
        

/data/CARD/PD/GP2/raw_genotypes/GP2_plink/204958240153_R02C01 doesnt exist
/data/CARD/PD/GP2/raw_genotypes/GP2_plink/204958240153_R03C01 doesnt exist
/data/CARD/PD/GP2/raw_genotypes/GP2_plink/204958240153_R04C01 doesnt exist
/data/CARD/PD/GP2/raw_genotypes/GP2_plink/204958240153_R05C01 doesnt exist
/data/CARD/PD/GP2/raw_genotypes/GP2_plink/204958240153_R01C01 doesnt exist
/data/CARD/PD/GP2/raw_genotypes/GP2_plink/204958240153_R08C01 doesnt exist
/data/CARD/PD/GP2/raw_genotypes/GP2_plink/204958240153_R06C01 doesnt exist
/data/CARD/PD/GP2/raw_genotypes/GP2_plink/204958240153_R07C01 doesnt exist
/data/CARD/PD/GP2/raw_genotypes/GP2_plink/204958240129_R08C01 doesnt exist
/data/CARD/PD/GP2/raw_genotypes/GP2_plink/204958240129_R04C01 doesnt exist
/data/CARD/PD/GP2/raw_genotypes/GP2_plink/204958240129_R07C01 doesnt exist
/data/CARD/PD/GP2/raw_genotypes/GP2_plink/204958240129_R06C01 doesnt exist
/data/CARD/PD/GP2/raw_genotypes/GP2_plink/204958240129_R01C01 doesnt exist
/data/CARD/PD/GP2/raw_gen

### Grab IPDGC-NG samples to return to Mei Rizig

In [6]:
ng_key = key.loc[key.study == 'IPDGCAF-NG']

for filename in ng_key.filename:
    code = filename.split('_')[0]
    grn = f'{idat_path}/{code}/{filename}_Grn.idat'
    red = f'{idat_path}/{code}/{filename}_Red.idat'
    if not os.path.isfile(grn) and os.path.isfile(red):
        print(f'{idat_path}/{code}/{filename} missing!')
    else:
        !module load google-cloud-sdk/359.0.0; gsutil cp {idat_path}/{code}/{filename}*.idat gs://ipdgcaf_ng/idats/
        

In [4]:
!module load google-cloud-sdk/359.0.0; gsutil ls gs://ipdgcaf_ng/idats/*.idat | wc -l

[-] Unloading python 3.8  ... 
[+] Loading python 3.7  ... 
[+] Loading google-cloud-sdk  359.0.0 

The following have been reloaded with a version change:
  1) python/3.8 => python/3.7

3220


In [9]:
pred_ancestry = pd.read_csv('/data/CARD/PD/GP2/genotypes/GP2/round2/clean/QC_metrics/pred_ancestry_labels.csv')
ng_key_merge = ng_key.merge(pred_ancestry, how='inner', left_on='GP2sampleID', right_on='IID')
for label in ng_key_merge.label.unique():
    
    geno_in = f'/data/CARD/PD/GP2/genotypes/GP2/round2/clean/GP2_round2_{label}'
    geno_out = f'/data/CARD/PD/GP2/genotypes/GP2/round2/clean/GP2_round2_{label}_IPDGCAF_NG'
    
    fam = pd.read_csv(
        f'{geno_in}.fam', 
        sep='\s+', 
        header=None, 
        names=['FID','IID','pat','mat','sex','pheno'], 
        usecols=['IID']
    )
    ng_samples = ng_key_merge.loc[ng_key_merge['GP2sampleID'].isin(fam['IID'])]
    ng_samples[['FID_x','GP2sampleID']].to_csv(f'{geno_out}.samples', sep='\t', header=False, index=False)
    
    cmd = f'\module load plink/2.3-alpha; \
plink2 \
--bfile {geno_in} \
--keep {geno_out}.samples \
--make-pgen \
--out {geno_out}'
    
    !{cmd}

!module load google-cloud-sdk/359.0.0; gsutil cp /data/CARD/PD/GP2/genotypes/GP2/round2/clean/GP2_round2_*_IPDGCAF_NG* gs://ipdgcaf_ng/clean/

[-] Unloading plink  1.9.0-beta4.4  on cn4257 
[+] Loading plink  2.3-alpha 

The following have been reloaded with a version change:
  1) plink/1.9.0-beta4.4 => plink/2.3-alpha

PLINK v2.00a3LM 64-bit Intel (11 Oct 2021)     www.cog-genomics.org/plink/2.0/
(C) 2005-2021 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /data/CARD/PD/GP2/genotypes/GP2/round2/clean/GP2_round2_AFR_IPDGCAF_NG.log.
Options in effect:
  --bfile /data/CARD/PD/GP2/genotypes/GP2/round2/clean/GP2_round2_AFR
  --keep /data/CARD/PD/GP2/genotypes/GP2/round2/clean/GP2_round2_AFR_IPDGCAF_NG.samples
  --make-pgen
  --out /data/CARD/PD/GP2/genotypes/GP2/round2/clean/GP2_round2_AFR_IPDGCAF_NG

Start time: Mon Jul 18 10:49:46 2022
1547809 MiB RAM detected; reserving 773904 MiB for main workspace.
Using up to 144 threads (change this with --threads).
1009 samples (377 females, 632 males; 1009 founders) loaded from
/data/CARD/PD/GP2/genotypes/GP2/round2/clean/GP2_round2_AFR.fam.
1888012 variants 

In [13]:
# now copy all IPDGCAF-NG samples (pre-qc included)
!module load google-cloud-sdk/359.0.0; gsutil cp /data/CARD/PD/GP2/raw_genotypes/GP2_QC/round2/GP2_QC_round2_IPDGCAF-NG.{bed,bim,fam} gs://ipdgcaf_ng/pre_qc/


[-] Unloading python 3.8  ... 
[+] Loading python 3.7  ... 
[+] Loading google-cloud-sdk  359.0.0 

The following have been reloaded with a version change:
  1) python/3.8 => python/3.7

Copying file:///data/CARD/PD/GP2/raw_genotypes/GP2_QC/round2/GP2_QC_round2_IPDGCAF-NG.bed [Content-Type=application/vnd.realvnc.bed]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite o

## Grab coriell vcfs for Zih-Hua

In [4]:
coriell_wgs = pd.read_csv('/data/CARD/PD/GP2/Coriell_to_wgs.csv')
coriell_key_merge = coriell_wgs.merge(key, how='left', on='GP2sampleID')
coriell_key_merge_clean = coriell_key_merge.loc[~coriell_key_merge.SentrixBarcode_A.isna()]
idat_path = '/data/CARD/PD/GP2/raw_genotypes/GP2_idats'

with open(f'{swarm_scripts_dir}/cp_coriell_vcfs.swarm', 'w') as f:
    for code in coriell_key_merge_clean.SentrixBarcode_A.astype(int).unique():
    #     print(code)
    #     !ls -lh {idat_path}/{code}/{code}_sorted.vcf.gz
        cp_cmd = f'gsutil cp {idat_path}/{code}/{code}_sorted.vcf.gz gs://gp2sequencing/neurobooster_vcfs/'
        f.write(f'{cp_cmd}\n')
f.close()


In [7]:
!swarm -f {swarm_scripts_dir}/cp_coriell_vcfs.swarm --logdir {swarm_scripts_dir}/logs --module google-cloud-sdk/359.0.0 --partition=norm
# !head {swarm_scripts_dir}/cp_coriell_vcfs.swarm

40945965


In [9]:
# coriell_key_merge.loc[coriell_key_merge.SentrixBarcode_A.isna()]
coriell_key_merge.to_csv('/data/CARD/PD/GP2/coriell_wgs_samples_key.csv', index=False)

### copy PROMO cohort from gcp to here

In [8]:
with open(f'{swarm_scripts_dir}/cp_PROMO.swarm', 'w') as f:
    cmd = f'gsutil cp -r gs://gp2sequencing/GP2_Complex_PROMO /data/CARD/PD/GP2/raw_genotypes/GP2_idats/'
    f.write(f'{cmd}')
f.close()
!swarm -f {swarm_scripts_dir}/cp_PROMO.swarm --logdir {swarm_scripts_dir}/logs --time=24:00:00 --module google-cloud-sdk/359.0.0 --partition=norm

44193162


In [14]:
# copy some specific samples for Zih-Hua
tmp_samples_list = [
    '204701870089_R02C01',
    '206102360027_R01C01',
    '204958250049_R04C01',
    '204701870063_R01C01',
    '205053520116_R01C01',
    '205746280054_R01C01',
    '205746280041_R02C01',
    '205763840013_R06C01',
]

for sample in tmp_samples_list:
    code = sample.split('_')[0]
    sample_path = f'/data/CARD/PD/GP2/raw_genotypes/GP2_idats/{code}/{sample}'
    !module load google-cloud-sdk/359.0.0; gsutil cp {sample_path}*.idat gs://gp2sequencing/neurobooster_vcfs/

[-] Unloading python 3.8  ... 
[+] Loading python 3.7  ... 
[+] Loading google-cloud-sdk  359.0.0 

The following have been reloaded with a version change:
  1) python/3.8 => python/3.7

Copying file:///data/CARD/PD/GP2/raw_genotypes/GP2_idats/204701870089/204701870089_R02C01_Grn.idat [Content-Type=application/octet-stream]...
Copying file:///data/CARD/PD/GP2/raw_genotypes/GP2_idats/204701870089/204701870089_R02C01_Red.idat [Content-Type=application/octet-stream]...
\ [2 files][ 65.2 MiB/ 65.2 MiB]                                                
Operation completed over 2 objects/65.2 MiB.                                     
[-] Unloading python 3.8  ... 
[+] Loading python 3.7  ... 
[+] Loading google-cloud-sdk  359.0.0 

The following have been reloaded with a version change:
  1) python/3.8 => python/3.7

Copying file:///data/CARD/PD/GP2/raw_genotypes/GP2_idats/206102360027/206102360027_R01C01_Grn.idat [Content-Type=application/octet-stream]...
Copying file:///data/CARD/PD/GP2/raw_

## grab UKM idats for Kin Mok

In [4]:
ukm_key = key.loc[key.study == 'IPDGCEA-UKM']
for filename in ukm_key.filename:
    code = filename.split('_')[0]
    grn = f'{idat_path}/{code}/{filename}_Grn.idat'
    red = f'{idat_path}/{code}/{filename}_Red.idat'
    if not os.path.isfile(grn) and os.path.isfile(red):
        print(f'{idat_path}/{code}/{filename} missing!')
    else:
        !module load google-cloud-sdk/359.0.0; gsutil cp {idat_path}/{code}/{filename}*.idat gs://ipdgcea_ukm/idats/

[-] Unloading python 3.8  ... 
[+] Loading python 3.7  ... 
[+] Loading google-cloud-sdk  359.0.0 

The following have been reloaded with a version change:
  1) python/3.8 => python/3.7

Copying file:///data/CARD/PD/GP2/raw_genotypes/GP2_idats/206377230004/206377230004_R01C01_Grn.idat [Content-Type=application/octet-stream]...
Copying file:///data/CARD/PD/GP2/raw_genotypes/GP2_idats/206377230004/206377230004_R01C01_Red.idat [Content-Type=application/octet-stream]...
\ [2 files][ 65.2 MiB/ 65.2 MiB]                                                
Operation completed over 2 objects/65.2 MiB.                                     
[-] Unloading python 3.8  ... 
[+] Loading python 3.7  ... 
[+] Loading google-cloud-sdk  359.0.0 

The following have been reloaded with a version change:
  1) python/3.8 => python/3.7

Copying file:///data/CARD/PD/GP2/raw_genotypes/GP2_idats/206377230004/206377230004_R02C01_Grn.idat [Content-Type=application/octet-stream]...
Copying file:///data/CARD/PD/GP2/raw_

In [7]:
pred_ancestry = pd.read_csv('/data/CARD/PD/GP2/genotypes/GP2/round2/clean/QC_metrics/pred_ancestry_labels.csv')
pred_ancestry.label.value_counts()

EUR    7364
EAS    1092
AFR    1027
AJ      971
AMR     556
AAC     462
SAS      92
FIN       5
Name: label, dtype: int64

## upload SYNAPS-KZ

In [16]:
kz_key = key.loc[key.study == 'SYNAPS-KZ']

for filename in kz_key.filename:
    code = filename.split('_')[0]
    grn = f'{idat_path}/{code}/{filename}_Grn.idat'
    red = f'{idat_path}/{code}/{filename}_Red.idat'
    if not os.path.isfile(grn) and os.path.isfile(red):
        print(f'{idat_path}/{code}/{filename} missing!')
    else:
        !module load google-cloud-sdk/359.0.0; gsutil cp {idat_path}/{code}/{filename}*.idat gs://synaps_kz/idats/

            
pred_ancestry = pd.read_csv('/data/CARD/PD/GP2/genotypes/GP2/round2/clean/QC_metrics/pred_ancestry_labels.csv')
kz_key_merge = kz_key.merge(pred_ancestry, how='inner', left_on='GP2sampleID', right_on='IID')
for label in kz_key_merge.label.unique():
    
    
    
    if label == 'AMR':
        geno_in = f'/data/CARD/PD/GP2/genotypes/GP2/round2/clean/GP2_round2_CAS'
        geno_out = f'/data/CARD/PD/GP2/genotypes/GP2/round2/clean/GP2_round2_CAS_SYNAPS_KZ'
    else:
        geno_in = f'/data/CARD/PD/GP2/genotypes/GP2/round2/clean/GP2_round2_{label}'
        geno_out = f'/data/CARD/PD/GP2/genotypes/GP2/round2/clean/GP2_round2_{label}_SYNAPS_KZ'
        
    fam = pd.read_csv(
        f'{geno_in}.fam', 
        sep='\s+', 
        header=None, 
        names=['FID','IID','pat','mat','sex','pheno'], 
        usecols=['IID']
    )
    
    kz_samples = kz_key_merge.loc[kz_key_merge['GP2sampleID'].isin(fam['IID'])]
    kz_samples[['FID_x','GP2sampleID']].to_csv(f'{geno_out}.samples', sep='\t', header=False, index=False)
    
    cmd = f'\module load plink/2.3-alpha; \
plink2 \
--bfile {geno_in} \
--keep {geno_out}.samples \
--make-pgen \
--out {geno_out}'
    
    !{cmd}
    



!module load google-cloud-sdk/359.0.0; gsutil cp /data/CARD/PD/GP2/genotypes/GP2/round2/clean/GP2_round2_*_SYNAPS_KZ* gs://synaps_kz/clean/

In [14]:
!module load google-cloud-sdk/359.0.0; gsutil ls gs://synaps_kz/idats | wc -l

[-] Unloading python 3.8  ... 
[+] Loading python 3.7  ... 
[+] Loading google-cloud-sdk  359.0.0 

The following have been reloaded with a version change:
  1) python/3.8 => python/3.7

668


## copy cleaned MDGAP-QSBB samples

In [6]:
# qsbb_key = key.loc[key.study=='MDGAP-QSBB']


# pred_ancestry = pd.read_csv('/data/CARD/PD/GP2/genotypes/GP2/round2/clean/QC_metrics/pred_ancestry_labels.csv')
# qsbb_key_merge = qsbb_key.merge(pred_ancestry, how='inner', left_on='GP2sampleID', right_on='IID')
# for label in qsbb_key_merge.label.unique():
    
    
#     geno_in = f'/data/CARD/PD/GP2/genotypes/GP2/round2/clean/GP2_round2_{label}'
#     geno_out = f'/data/CARD/PD/GP2/genotypes/GP2/round2/clean/GP2_round2_{label}_MDGAP_QSBB'
        
#     fam = pd.read_csv(
#         f'{geno_in}.fam', 
#         sep='\s+', 
#         header=None, 
#         names=['FID','IID','pat','mat','sex','pheno'], 
#         usecols=['IID']
#     )
    
#     samples_ = qsbb_key_merge.loc[qsbb_key_merge['GP2sampleID'].isin(fam['IID'])]
#     samples_[['FID_x','GP2sampleID']].to_csv(f'{geno_out}.samples', sep='\t', header=False, index=False)
    
#     cmd = f'\module load plink/2.3-alpha; \
# plink2 \
# --bfile {geno_in} \
# --keep {geno_out}.samples \
# --make-pgen \
# --out {geno_out}'
    
#     !{cmd}


!module load google-cloud-sdk/359.0.0; gsutil cp /data/CARD/PD/GP2/genotypes/GP2/round2/clean/GP2_round2_*_MDGAP_QSBB* gs://mdgap_qsbb/clean/

[-] Unloading python 3.8  ... 
[+] Loading python 3.7  ... 
[+] Loading google-cloud-sdk  359.0.0 

The following have been reloaded with a version change:
  1) python/3.8 => python/3.7

Copying file:///data/CARD/PD/GP2/genotypes/GP2/round2/clean/GP2_round2_AJ_MDGAP_QSBB.log [Content-Type=text/plain]...
Copying file:///data/CARD/PD/GP2/genotypes/GP2/round2/clean/GP2_round2_AJ_MDGAP_QSBB.pgen [Content-Type=application/octet-stream]...
Copying file:///data/CARD/PD/GP2/genotypes/GP2/round2/clean/GP2_round2_AJ_MDGAP_QSBB.psam [Content-Type=application/octet-stream]...
Copying file:///data/CARD/PD/GP2/genotypes/GP2/round2/clean/GP2_round2_AJ_MDGAP_QSBB.pvar [Content-Type=application/octet-stream]...
/ [4 files][ 58.2 MiB/ 58.2 MiB]    1.1 MiB/s                                   
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about 

In [17]:
!module load google-cloud-sdk/359.0.0; gsutil cp /data/CARD/PD/GP2/raw_genotypes/GP2_QC/round2/GP2_QC_round2_MDGAP-QSBB.* gs://mdgap_qsbb/raw_genotypes/

[-] Unloading python 3.8  ... 
[+] Loading python 3.7  ... 
[+] Loading google-cloud-sdk  359.0.0 

The following have been reloaded with a version change:
  1) python/3.8 => python/3.7

Copying file:///data/CARD/PD/GP2/raw_genotypes/GP2_QC/round2/GP2_QC_round2_MDGAP-QSBB.bed [Content-Type=application/vnd.realvnc.bed]...
Copying file:///data/CARD/PD/GP2/raw_genotypes/GP2_QC/round2/GP2_QC_round2_MDGAP-QSBB.bim [Content-Type=application/octet-stream]...
Copying file:///data/CARD/PD/GP2/raw_genotypes/GP2_QC/round2/GP2_QC_round2_MDGAP-QSBB.fam [Content-Type=application/octet-stream]...
Copying file:///data/CARD/PD/GP2/raw_genotypes/GP2_QC/round2/GP2_QC_round2_MDGAP-QSBB.hh [Content-Type=text/plain]...
- [4 files][114.8 MiB/114.8 MiB]  773.9 KiB/s                                   
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
abo

In [31]:
labels = ['AAC','AFR','AJ','AMR','EAS','EUR','FIN','SAS','CAS']
basepath = '/data/CARD/PD/GP2/genotypes/GP2/round2/clean/GP2_round2'

# create '.related' file for CAS
slist_path = f'{basepath}_CAS_release2.samples'
related_path = f'{basepath}_AMR.related'
slist = pd.read_csv(slist_path, sep='\t', header=None, names=['FID','IID'])
rlist = pd.read_csv(related_path, sep='\t')
rlist.drop(columns=['FID'], inplace=True)

sr_merge = slist.merge(rlist, how='left', on='IID')
sr_merge.loc[~sr_merge.status.isna()].to_csv(f'{basepath}_CAS.related', index=False, sep='\t')



# now, create related files for release 2 samples only (per-ancestry)
for label in labels:
    rout = f'{basepath}_{label}_release2.related'
    slist_path = f'{basepath}_{label}_release2.samples'
    related_path = f'{basepath}_{label}.related'
    slist = pd.read_csv(slist_path, sep='\t', header=None, names=['FID','IID'])
    rlist = pd.read_csv(related_path, sep='\t')
    slist.drop(columns=['FID'], inplace=True)
    
    r_df_out = rlist.loc[rlist['IID'].isin(slist['IID'])]
    r_df_out.to_csv(rout, sep='\t', header=True, index=False)

# push relateds lists to release2 bucket
!module load google-cloud-sdk/359.0.0; gsutil cp /data/CARD/PD/GP2/genotypes/GP2/round2/clean/GP2_round2_{AAC,AFR,AJ,AMR,CAS,EAS,EUR,FIN,SAS}_release2.related gs://gp2tier2/release2_06052022/meta_data/
    
    

# process idats
- convert idat --> ped --> bed
- merge
- add phenotype, sex, and update ids to GP2 ids

In [ ]:
# create ped
with open(f'{swarm_scripts_dir}/idat_to_ped.swarm', 'w') as f:
    
    for code in key.SentrixBarcode_A.unique():
        
        idat_to_ped_cmd = f'\
{iaap} gencall \
{bpm} \
{egt} \
{raw_plink_path}/ \
-f {idat_path}/{code} \
-p \
-t 8'
        
        f.write(f'{idat_to_ped_cmd}\n')
f.close()

# run swarm
shell_do(f'swarm -f {swarm_scripts_dir}/idat_to_ped.swarm -g 32 -t 16 --time=10:00:00 --logdir {swarm_scripts_dir}/logs --gres=lscratch:20 --partition=norm')

In [ ]:
# copy map file to match name of each ped
map_file = f'{raw_plink_path}/NeuroBooster_20042459_A2.map'
for filename in key.filename:
    ped = f'{raw_plink_path}/{filename}.ped'
    out_map = f'{raw_plink_path}/{filename}.map'
    if os.path.isfile(ped):
        shutil.copyfile(src=map_file, dst=out_map)
    else:
        print(f'{ped} does not exist!')
        print(f'{out_map} creation cancelled')

In [ ]:
# create bed
with open(f'{swarm_scripts_dir}/make_bed.swarm', 'w') as f:
    for filename in key.filename:
        ped = f'{raw_plink_path}/{filename}'
        make_bed_cmd = f'\
plink \
--file {ped} \
--make-bed \
--out {raw_plink_path}/{filename}'

        f.write(f'{make_bed_cmd}\n')
f.close()

# run swarm
shell_do(f'swarm -f {swarm_scripts_dir}/make_bed.swarm -g 64 -t 32 --time=10:00:00 --logdir {swarm_scripts_dir}/logs --gres=lscratch:20 --partition=norm')

In [ ]:
# write plink merge command and print missing
with open(f"{raw_plink_path}/merge_bed.list", 'w') as f:
    for filename in key.filename:
        bed = f'{raw_plink_path}/{filename}'
        if os.path.isfile(f'{bed}.bed'):
            f.write(f'{bed}\n')
        else:
            print(f'{bed} does not exist!!!')
f.close()

with open(f"{swarm_scripts_dir}/merge.swarm", 'w') as f:

    plink_merge_cmd = f'\
plink \
--merge-list {raw_plink_path}/merge_bed.list \
--update-ids {clin_dir}/update_ids.txt \
--make-bed \
--out {raw_plink_path}/GP2_merge'
    f.write(f"{plink_merge_cmd}")
f.close()

# run swarm
shell_do(f'swarm -f {swarm_scripts_dir}/merge.swarm -g 64 -t 32 --time=10:00:00 --logdir {swarm_scripts_dir}/logs --gres=lscratch:20 --partition=norm')


In [ ]:
# merge bed files to single bed
shell_do(f'plink --bfile {raw_plink_path}/GP2_merge --pheno {clin_dir}/update_pheno.txt --update-sex {clin_dir}/update_sex.txt --make-bed --out {geno_path}'

# GenoTools

## qc pipeline

In [ ]:
with open(f'{swarm_scripts_dir}/run_qc_pipeline.swarm','w') as f:
    run_pipeline = f'python3 /data/vitaled2/GenoTools/run_qc_pipeline.py --geno {geno_path} --ref {ref_panel} --ref_labels {ref_labels} --out {out_path}'
    f.write(f'{run_pipeline}\n')
f.close()

# run swarm
shell_do(f'!swarm -f {swarm_scripts_dir}/run_qc_pipeline.swarm -g 64 -t 32 --time=10:00:00 --logdir {swarm_scripts_dir}/logs --module python/3.8 --gres=lscratch:20 --partition=norm')

In [5]:
impute_genos_list = [x.split('.')[0] for x in glob.glob(f'{out_dir}/*.bed')]
impute_labels_list = [x.split('/')[-1].replace('.bed','').split('_')[-1] for x in impute_genos_list]

## imputation pipeline

### imputation data prep
- bim checked with topmed reference panel
- hg38

In [9]:
for geno, label in zip(impute_genos_list, impute_labels_list):
    label_outdir = f'{imputed_out_dir}/{label}'
    impute_prep_outdir = f'{gp2_qc_path}/round1/{label}'
    impute_prep_geno = f'{impute_prep_outdir}/{label}'

    os.makedirs(f'{label_outdir}', exist_ok=True)
    os.makedirs(f'{impute_prep_outdir}', exist_ok=True)
    
    impute_data = impute_data_prep(geno, impute_prep_geno, topmed_ref_panel, check_bim_pl)


# would like to run this way. need to figure out for swarm jobs
#     run_pipeline = f'python3 ../run_imputation_pipeline.py --geno {geno} --temp {label_temp} --token {config.api_key} --ref_panel {topme_ref_panel} --check_bim_pl {check_bim_pl} --out {label_outdir}/'

### submit imputation server jobs
- topmed server
- save job ids

In [ ]:
job_id_df = pd.DataFrame({'job_id':list(),'label':list(),'path':list()})
job_id_df.to_csv(f'{imputed_out_dir}/GP2_QC_round1_impute_job_ids.csv')

token = config.api_key
password = 'imputer'

for geno, label in zip(impute_genos_list, impute_labels_list):
    label_outdir = f'{imputed_out_dir}/{label}'
    impute_prep_outdir = f'{gp2_qc_path}/round1/{label}'
    impute_prep_geno = f'{impute_prep_outdir}/{label}'

    vcf_list = [f'{impute_prep_geno}_pre_impute_chr{str(i)}.vcf.gz' for i in range(1,24)]
    
    print(f'LABEL: {label}')
    
    job_json = submit_job(vcf_list, password=password, token=token)
    job_id = job_json['id']

    job_id_df_tmp = pd.DataFrame({'job_id': [job_id], 'label': [label], 'path': [geno]})
    job_id_df_tmp.to_csv(f'{imputed_out_dir}/GP2_QC_round1_impute_job_ids.csv', mode='a', header=False, index=False)



### pull imputed data from server

In [24]:
job_ids = pd.read_csv(f'{imputed_out_dir}/GP2_QC_round1_impute_job_ids.csv', index_col=0)
job_ids = job_ids.reset_index()

In [10]:
# pull completed jobs

for i,job_id in enumerate(job_ids.job_id):
    label = job_ids.loc[i,'label']
    path = job_ids.loc[i,'path']
    outpath = path.replace('clean', 'imputed').replace('GP2_round1_','') + '/'
    pull_imputed_data(out_path=outpath, token=token, job_id=job_id, password='imputer')

## clean up imputed data

- unzip
- convert to pgen
- keep only snps with minor allele count > 10 minor allele count and r2 > 0.3

In [8]:
# unzip for each chromosome in each cohort
with open(f'{swarm_scripts_dir}/unzip_imputed.swarm', 'w') as f:
    for i, path in enumerate(job_ids.path):
        label = job_ids.loc[i,'label']
        outpath = path.replace('clean', 'imputed').replace('GP2_round1_','')
        for chrom in chroms:
            zipfile = f'chr_{chrom}.zip'
            unzip_cmd = f'cd {outpath}; unzip -P imputer {zipfile}'
            f.write(f'{unzip_cmd}\n')
f.close()

shell_do(f'swarm -f {swarm_scripts_dir}/unzip_imputed.swarm -g 64 -t 16 --time=10:00:00 --logdir {swarm_scripts_dir}/logs --gres=lscratch:20 --partition=norm')

In [ ]:
# prune and convert to pgen
with open(f'{swarm_scripts_dir}/imputed_make_pgen.swarm', 'w') as f:
    for i, path in enumerate(job_ids.path):
        label = job_ids.loc[i,'label']
        outpath = path.replace('clean', 'imputed').replace('GP2_round1_','')
        for chrom in chroms:
            vcf = f'{outpath}/chr{chrom}.dose.vcf.gz'
            pgen_out = f'{outpath}/chr{chrom}_{label}'
            cmd = f"plink2 --vcf {vcf} 'dosage=HDS' --mac 10 --minimac3-r2-filter 0.3 --make-pgen --out {pgen_out}"
            f.write(f'{cmd}\n')
f.close()

shell_do(f'swarm -f {swarm_scripts_dir}/imputed_make_pgen.swarm -g 64 -t 32 --time=10:00:00 --logdir {swarm_scripts_dir}/logs --module plink/2.3-alpha --gres=lscratch:20 --partition=norm')

### clean up pgens for release
- update ids (returned as FID_IID as IID in plink bed)
- add phenotype
- add sex
- extract only release1 samples (BCM, UMD, CORIELL)

In [8]:
master_key = pd.read_csv(f'{clin_dir}/key_merge/round1/GP2_master_key_full.txt', sep='\t')
master_key[master_key.study.isin(['BCM','UMD','CORIELL'])].to_csv(f'/data/CARD/PD/GP2/genotypes/GP2/round1/master_key_release1.csv', index=False)

In [30]:
# fix ids from imputation server

for i, path in enumerate(job_ids.path):
        
    label = job_ids.loc[i,'label']
    outpath = path.replace('clean', 'imputed').replace('GP2_round1_','')
    samplefile = f'{outpath}/{label}'
    pgen_out = f'{outpath}/chr1_{label}'
    if os.path.isfile(f'{pgen_out}.psam'):
        psam = pd.read_csv(f'{pgen_out}.psam', sep='\s+')
        psam.loc[:,'IID_new'] = psam.loc[:,'#IID'].str.replace('0_','',n=1)
        psam_key = psam.merge(master_key, left_on='IID_new', right_on='GP2sampleID', how='left')
        psam_key.loc[:,'sex'] = psam_key.loc[:,'sex'].astype(int).astype(str)
        psam_key.loc[:,'pheno'] = psam_key.loc[:,'pheno'].astype(int).astype(str)
        release1_samples = psam_key.loc[psam_key.study.isin(['BCM','UMD','CORIELL'])]
        release1_samples_final = release1_samples[release1_samples.pheno!='-9']

        psam[['#IID','IID_new']].to_csv(f'{samplefile}.update_ids', sep='\t', header=False, index=False)
        psam_key[['FID','IID_new','sex']].to_csv(f'{samplefile}.sex', sep='\t', header=False, index=False)
        psam_key[['IID_new','pheno']].to_csv(f'{samplefile}.pheno', sep='\t', header=False, index=False)
        release1_samples_final['IID_new'].to_csv(f'{samplefile}_release1.samples', sep='\t', header=False, index=False)
        print(label, release1_samples_final.shape[0])
    else:
        print(f'{pgen_out} does not exist!')

# write commands to change ids, add pheno, sex, and keep only release1 samples
with open(f'{swarm_scripts_dir}/release1_prep.swarm', 'w') as f:
    
    for i, path in enumerate(job_ids.path):
        
        label = job_ids.loc[i,'label']
        outpath = path.replace('clean', 'imputed').replace('GP2_round1_','')
        samplefile = f'{outpath}/{label}'

        for chrom in chroms:
        
            pgen_out = f'{outpath}/chr{chrom}_{label}'
            if os.path.isfile(f'{pgen_out}.psam'):

                update_ids_cmd = f'plink2 --pfile {pgen_out} --update-ids {samplefile}.update_ids --make-pgen --out {pgen_out}_update_ids'
                release_cmd = f"plink2 --pfile {pgen_out}_update_ids --keep {samplefile}_release1.samples --pheno {samplefile}.pheno 'iid-only' --update-sex {samplefile}.sex --make-pgen --out {pgen_out}_release1"

                f.write(f'{update_ids_cmd}; {release_cmd}\n')
            
f.close()

# run swarm
!swarm -f {swarm_scripts_dir}/release1_prep.swarm -g 64 -t 32 --time=10:00:00 --logdir {swarm_scripts_dir}/logs --module plink/2.3-alpha --gres=lscratch:20 --partition=norm
# shell_do(f'swarm -f {swarm_scripts_dir}/release1_prep.swarm -g 64 -t 32 --time=10:00:00 --logdir {swarm_scripts_dir}/logs --module plink/2.3-alpha --gres=lscratch:20 --partition=norm')

EAS 46
/data/CARD/PD/GP2/genotypes/GP2/round1/imputed/FIN/chr1_FIN does not exist!
SAS 47
AAC 162
AJ 520
EUR 3919
AMR 214
28559309


In [31]:
# now get sample counts per ancestry
total_count = 0
for i, path in enumerate(job_ids.path):
        label = job_ids.loc[i,'label']
        outpath = path.replace('clean', 'imputed').replace('GP2_round1_','')
        psam_path = f'{outpath}/chr1_{label}_release1.psam'
        if os.path.exists(psam_path):
            psam = pd.read_csv(psam_path, sep='\s+')
            print(label,psam.shape[0])
            total_count += psam.shape[0]
        else:
            pass
print(f'total: {total_count}')

EAS 46
SAS 47
AAC 162
AJ 520
EUR 3919
AMR 214
total: 4908


### copy imputed genotypes to bucket

In [32]:
for i, path in enumerate(job_ids.path):
        
        label = job_ids.loc[i,'label']
        outpath = path.replace('clean', 'imputed').replace('GP2_round1_','')
        samplefile = f'{outpath}/{label}'

        for chrom in chroms:
            pgen_out = f'{outpath}/chr{chrom}_{label}_release1'
            if os.path.exists(f'{pgen_out}.pgen'):
                !module load google-cloud-sdk/359.0.0; gsutil cp {pgen_out}* gs://gp2_tier2/release1_29112021/imputed_genotypes/{label}


[-] Unloading python 3.7  ... 
[+] Loading python 3.7  ... 
[+] Loading google-cloud-sdk  359.0.0 
Copying file:///data/CARD/PD/GP2/genotypes/GP2/round1/imputed/EAS/chr1_EAS_release1.log [Content-Type=text/plain]...
Copying file:///data/CARD/PD/GP2/genotypes/GP2/round1/imputed/EAS/chr1_EAS_release1.pgen [Content-Type=application/octet-stream]...
Copying file:///data/CARD/PD/GP2/genotypes/GP2/round1/imputed/EAS/chr1_EAS_release1.psam [Content-Type=application/octet-stream]...
Copying file:///data/CARD/PD/GP2/genotypes/GP2/round1/imputed/EAS/chr1_EAS_release1.pvar [Content-Type=application/octet-stream]...
/ [4 files][ 94.6 MiB/ 94.6 MiB]                                                
Operation completed over 4 objects/94.6 MiB.                                     
[-] Unloading python 3.7  ... 
[+] Loading python 3.7  ... 
[+] Loading google-cloud-sdk  359.0.0 
Copying file:///data/CARD/PD/GP2/genotypes/GP2/round1/imputed/EAS/chr2_EAS_release1.log [Content-Type=text/plain]...
Copying f

## grab release1 samples from raw qc'd genotypes and push to bucket

In [21]:
# master_key
for geno, label in zip(impute_genos_list, impute_labels_list):
    label_outdir = f'{out_dir}/{label}'
    release1_geno = f'{geno}_release1'
    related_path = f'{geno}.related'
    related = pd.read_csv(f'{related_path}', sep='\t')

    fam = pd.read_csv(f'{geno}.fam', sep='\s+', header=None, names=['FID','IID', 'pat','mat','sex','pheno'], dtype={'pheno':str,'sex':str})
    fam_key_merge = fam.merge(master_key, how='left',left_on='IID', right_on='GP2sampleID')
    fam_key_release1 = fam_key_merge[fam_key_merge.study.isin(['BCM','UMD','CORIELL'])]

    fam_key_release1_final = fam_key_release1[fam_key_release1.pheno_x!='-9']
    related_merge = related.merge(fam_key_release1_final, how='inner', left_on=['FID','IID'], right_on=['FID_x','IID_x'])

#     print(fam_key_release1_final.pheno_x.value_counts())
    fam_key_release1[['FID_x','IID_x']].to_csv(f'{release1_geno}.samples', sep='\t', header=False, index=False)
    related_merge[['FID','IID','status']].to_csv(f'{release1_geno}.related', sep='\t', header=True, index=False)
    
    
    
    
    plink_keep_cmd = f'module load plink/2.3-alpha; \
plink2 \
--bfile {geno} \
--keep {release1_geno}.samples \
--make-pgen \
--out {release1_geno}'
    
#     !{plink_keep_cmd}
#     !module load google-cloud-sdk/359.0.0; gsutil cp {release1_geno}* gs://gp2_tier2/release1_29112021/raw_genotypes/
    !module load google-cloud-sdk/359.0.0; gsutil cp {release1_geno}.related gs://gp2_tier2/release1_29112021/raw_genotypes/


[-] Unloading python 3.7  ... 
[+] Loading python 3.7  ... 
[+] Loading google-cloud-sdk  359.0.0 
Copying file:///data/CARD/PD/GP2/genotypes/GP2/round1/clean/GP2_round1_EAS_release1.related [Content-Type=application/octet-stream]...
/ [1 files][   15.0 B/   15.0 B]                                                
Operation completed over 1 objects/15.0 B.                                       
[-] Unloading python 3.7  ... 
[+] Loading python 3.7  ... 
[+] Loading google-cloud-sdk  359.0.0 
Copying file:///data/CARD/PD/GP2/genotypes/GP2/round1/clean/GP2_round1_FIN_release1.related [Content-Type=application/octet-stream]...
/ [1 files][   15.0 B/   15.0 B]                                                
Operation completed over 1 objects/15.0 B.                                       
[-] Unloading python 3.7  ... 
[+] Loading python 3.7  ... 
[+] Loading google-cloud-sdk  359.0.0 
Copying file:///data/CARD/PD/GP2/genotypes/GP2/round1/clean/GP2_round1_SAS_release1.related [Content-Type=a

In [41]:
# get case/control counts for genotracker
for geno, label in zip(impute_genos_list, impute_labels_list):
    label_outdir = f'{out_dir}/{label}'
    release1_geno = f'{geno}_release1'

    fam = pd.read_csv(f'{geno}.fam', sep='\s+', header=None, names=['FID','IID', 'pat','mat','sex','pheno'], dtype={'pheno':str,'sex':str})
    fam_key_merge = fam.merge(master_key, how='left',left_on='IID', right_on='GP2sampleID')
    fam_key_release1 = fam_key_merge[fam_key_merge.study.isin(['BCM','UMD','CORIELL'])]
    fam_key_release1 = fam_key_release1[fam_key_release1.pheno_x!='-9']
    
    print(f'Case/Control Count {label}')
    print(fam_key_release1.pheno_x.value_counts())
    print()

Case/Control Count EAS
2    35
1    11
Name: pheno_x, dtype: int64

Case/Control Count FIN
2    5
1    1
Name: pheno_x, dtype: int64

Case/Control Count SAS
2    38
1     9
Name: pheno_x, dtype: int64

Case/Control Count AAC
1    81
2    81
Name: pheno_x, dtype: int64

Case/Control Count AJ
2    396
1    124
Name: pheno_x, dtype: int64

Case/Control Count EUR
2    2732
1    1187
Name: pheno_x, dtype: int64

Case/Control Count AMR
2    152
1     62
Name: pheno_x, dtype: int64



In [3]:
# get IIDs of missing phenotype samples
mis_pheno_df = pd.DataFrame()
for geno, label in zip(impute_genos_list, impute_labels_list):
    label_outdir = f'{out_dir}/{label}'
    release1_geno = f'{geno}_release1'

    fam = pd.read_csv(f'{geno}.fam', sep='\s+', header=None, names=['FID','IID', 'pat','mat','sex','pheno'])
    fam_key_merge = fam.merge(master_key, how='left',left_on='IID', right_on='GP2sampleID')
    fam_key_release1 = fam_key_merge[fam_key_merge.study.isin(['BCM','UMD','CORIELL'])]
    
    fam_mis = fam.loc[fam.pheno==-9]
    mis_pheno_df = mis_pheno_df.append(fam_mis)
mis_pheno_df.to_csv(f'{clin_dir}/missing_phenos.csv', header=True, index=False)

In [6]:
# fix master key to remove unnecessary columns
key_old = pd.read_csv('/data/CARD/PD/GP2/genotypes/GP2/round1/master_key_release1.csv')
key_final = key_old.drop(columns=['Sample_ID','SentrixBarcode_A','SentrixPosition_A','sample_id','FID','IID','clinical_id'])
key_final.to_csv('/data/CARD/PD/GP2/genotypes/GP2/round1/master_key_release1_final.csv')

### copy clinical master key and qc meta data to bucket

In [7]:
!module load google-cloud-sdk/359.0.0; gsutil cp /data/CARD/PD/GP2/genotypes/GP2/round1/master_key_release1_final.csv gs://gp2_tier2/release1_29112021/clinical_data/
!module load google-cloud-sdk/359.0.0; gsutil cp {QC_metrics_path} gs://gp2_tier2/release1_29112021/meta_data/

[-] Unloading python 3.7  ... 
[+] Loading python 3.7  ... 
[+] Loading google-cloud-sdk  359.0.0 
Copying file:///data/CARD/PD/GP2/genotypes/GP2/round1/master_key_release1_final.csv [Content-Type=text/csv]...
/ [1 files][955.9 KiB/955.9 KiB]                                                
Operation completed over 1 objects/955.9 KiB.                                    


## look at qc metrics

In [5]:
QC_metrics_path = f'{out_path}.QC.metrics.h5'

metrics_df = pd.read_hdf(QC_metrics_path, key='QC')
pruned_samples = pd.read_hdf(QC_metrics_path, key='pruned_samples')
ancestry_counts_df = pd.read_hdf(QC_metrics_path, key='ancestry_counts')
pred_ancestry_labels = pd.read_hdf(QC_metrics_path, key='ancestry_labels')
conf_mat_df = pd.read_hdf(QC_metrics_path, key='confusion_matrix', index=True)
ref_pcs = pd.read_hdf(QC_metrics_path, key='ref_pcs')
projected_pcs = pd.read_hdf(QC_metrics_path, key='projected_pcs')
total_umap = pd.read_hdf(QC_metrics_path, key='total_umap')
ref_umap = pd.read_hdf(QC_metrics_path, key='ref_umap')
new_samples_umap = pd.read_hdf(QC_metrics_path, key='new_samples_umap')

ValueError: unsupported pickle protocol: 5

In [ ]:
# pruned_samples[pruned_samples.step=='callrate_prune']
pruned_samples_pheno = pruned_samples.merge(pheno_key, on='IID', how='left')
pruned_samples_pheno

In [ ]:
for cohort in pruned_samples_pheno.Cohort_name.unique():
    print(cohort)
    print(pruned_samples_pheno[pruned_samples_pheno.Cohort_name == cohort].step.value_counts())
    print()

In [ ]:
pred_ancestry_labels_pheno = pred_ancestry_labels.merge(pheno, left_on='IID', right_on='GP2SampleID', how='left')

In [ ]:
for cohort in pred_ancestry_labels_pheno.Cohort_name.unique():
    print(cohort)
    print(pred_ancestry_labels_pheno[pred_ancestry_labels_pheno.Cohort_name == cohort].label.value_counts())
    print()

#### Create key for final GP2 ancestry list in release 2

In [19]:
labels = ['AAC','AFR','AJ','AMR','CAS','EAS','EUR','FIN','SAS']
out_samples_df = pd.DataFrame()
for label in labels:
    samples_list_path = f'/data/CARD/PD/GP2/genotypes/GP2/round2/clean/GP2_round2_{label}_release2.samples'
    samples_list = pd.read_csv(samples_list_path, sep='\t', header=None, names=['FID','IID'])
    samples_list.loc[:,'label'] = label
    out_samples_df = pd.concat([out_samples_df, samples_list], ignore_index=True)

out_samples_df.to_csv('/data/CARD/PD/GP2/genotypes/GP2/round2/clean/GP2_release2_full_sample_ancestry.txt', sep='\t', index=False)

In [6]:
# !gcloud config set gp2-release-terra

/bin/bash: gcloud: command not found


In [20]:
!head /data/CARD/PD/GP2/genotypes/GP2/round2/clean/GP2_release2_full_sample_ancestry.txt

FID	IID	label
0	BCM_000286_s1	AAC
0	UMD_000397_s1	AAC
0	BCM_000562_s1	AAC
0	CORIELL_000674_s1	AAC
0	UMD_000118_s1	AAC
0	BCM_000287_s1	AAC
0	BCM_000150_s1	AAC
0	CORIELL_000670_s1	AAC
0	CORIELL_001231_s1	AAC
